# Advent of Code 2024

### --- Day 18: RAM Run ---

You and The Historians look a lot more pixelated than you remember. You're inside a computer at the North Pole!

Just as you're about to check out your surroundings, a program runs up to you. "This region of memory isn't safe! The User misunderstood what a pushdown automaton is and their algorithm is pushing whole bytes down on top of us! Run!"

The algorithm is fast - it's going to cause a byte to fall into your memory space once every nanosecond! Fortunately, you're faster, and by quickly scanning the algorithm, you create a list of which bytes will fall (your puzzle input) in the order they'll land in your memory space.

Your memory space is a two-dimensional grid with coordinates that range from 0 to 70 both horizontally and vertically. However, for the sake of example, suppose you're on a smaller grid with coordinates that range from 0 to 6 and the following list of incoming byte positions:

```
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
```

Each byte position is given as an X,Y coordinate, where X is the distance from the left edge of your memory space and Y is the distance from the top edge of your memory space.

You and The Historians are currently in the top left corner of the memory space (at 0,0) and need to reach the exit in the bottom right corner (at 70,70 in your memory space, but at 6,6 in this example). You'll need to simulate the falling bytes to plan out where it will be safe to run; for now, simulate just the first few bytes falling into your memory space.

As bytes fall into your memory space, they make that coordinate corrupted. Corrupted memory coordinates cannot be entered by you or The Historians, so you'll need to plan your route carefully. You also cannot leave the boundaries of the memory space; your only hope is to reach the exit.

In the above example, if you were to draw the memory space after the first 12 bytes have fallen (using . for safe and # for corrupted), it would look like this:

```
...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....
```

You can take steps up, down, left, or right. After just 12 bytes have corrupted locations in your memory space, the shortest path from the top left corner to the exit would take 22 steps. Here (marked with O) is one such path:

```
OO.#OOO
.O#OO#O
.OOO#OO
...#OO#
..#OO#.
.#.O#..
#.#OOOO
```

Simulate the first kilobyte (1024 bytes) falling onto your memory space. Afterward, what is the minimum number of steps needed to reach the exit?



In [27]:
import sys
import math
import heapq
from collections import deque
size = 71
limit = 1024
directions = [
    (0,1),(-1,0), (0,-1), (1,0)
]

# Parse the input file
with open('input/day_18.txt') as f:
    lines = f.read().splitlines()

# print(lines)

def valid(c):
    return  c[0]>=0 and c[0]<size and c[1]>=0 and c[1]<size
def printa(p):
    for i in range(size):
        for j in range(size):
            if (i,j) in p:
                c = '#'
            else:
                c = '.'
            print(c, end='')
        print()

def bfs(current, target, points):
    q = deque([(current, 0, [])])
    values = []
    for i in range(size):
        values.append([])
        for j in range(size):
            values[i].append(math.inf)

    result = math.inf
    while q:
        if len(q)%1000==0:
            print(len(q))
        n,v,path = q.popleft()
        if n==target:
            result = min(result, v)
            continue
        if values[n[0]][n[1]]<v:
            continue
        values[n[0]][n[1]] = v
        visited = path + [n]
        for d in directions:
            u = (n[0]+d[0], n[1]+d[1])
            if u in visited:
                continue
            if not valid(u) or u in points:
                continue
            q.append((u, v+1, visited))
    return result

def calc(i,j,ti,tj):
    return abs(i-ti)+abs(j-tj)


def astar(src, dst, points):
    closed = [[False for _ in range(size)] for _ in range(size)]
    # parent i, parent j, (g+h), g (dist src to this), h (dist to dst using heuristic)
    details =[[(0,0,math.inf,math.inf, 0) for _ in range(size)] for _ in range(size)]

    i,j=src
    details[i][j]=(i,j,0,0,0)
    olist = []
    heapq.heappush(olist, (0.0, i,j))

    found = False

    while len(olist)>0:
        _,i,j = heapq.heappop(olist)
        closed[i][j]=True
        for d in directions:
            ni = i+d[0]
            nj = j+d[1]

            if not valid((ni,nj)) or (ni,nj) in points or closed[ni][nj]:
                continue
            g = details[i][j][3] +1
            if (ni,nj)==dst:
                details[ni][nj]=(i,j,g,g,0)
                return True,g
            else:
                h = calc(i,j,*dst)
                f = g+h
                cf=details[ni][nj][2]
                if cf==math.inf or cf>f:
                    heapq.heappush(olist, (f, ni,nj))
                    details[ni][nj]=(i,j,f,g,h)
    return False,0
points = []

for line in lines:
    x,y = line.split(',')
    points.append((int(x),int(y)))

points = points[:limit]
#print(points)
#printa(points)
# print(bfs((0,0), (size-1,size-1), points))
print(astar((0,0), (size-1, size-1), points))
#2390 to high

(True, 338)


### --- Part Two ---

The Historians aren't as used to moving around in this pixelated universe as you are. You're afraid they're not going to be fast enough to make it to the exit before the path is completely blocked.

To determine how fast everyone needs to go, you need to determine the first byte that will cut off the path to the exit.

In the above example, after the byte at 1,1 falls, there is still a path to the exit:

```
O..#OOO
O##OO#O
O#OO#OO
OOO#OO#
###OO##
.##O###
#.#OOOO
```

However, after adding the very next byte (at 6,1), there is no longer a path to the exit:

```
...#...
.##..##
.#..#..
...#..#
###..##
.##.###
#.#....
```

So, in this example, the coordinates of the first byte that prevents the exit from being reachable are 6,1.

Simulate more of the bytes that are about to corrupt your memory space. What are the coordinates of the first byte that will prevent the exit from being reachable from your starting position? (Provide the answer as two integers separated by a comma with no other characters.)

In [28]:
import sys
import math
import heapq
from collections import deque
size = 71
limit = 1024
directions = [
    (0,1),(-1,0), (0,-1), (1,0)
]

# Parse the input file
with open('input/day_18.txt') as f:
    lines = f.read().splitlines()

def valid(c):
    return  c[0]>=0 and c[0]<size and c[1]>=0 and c[1]<size
def printa(p):
    for i in range(size):
        for j in range(size):
            if (i,j) in p:
                c = '#'
            else:
                c = '.'
            print(c, end='')
        print()

def bfs(current, target, points):
    q = deque([(current, 0, [])])
    values = []
    for i in range(size):
        values.append([])
        for j in range(size):
            values[i].append(math.inf)

    result = math.inf
    while q:
        if len(q)%1000==0:
            print(len(q))
        n,v,path = q.popleft()
        if n==target:
            result = min(result, v)
            continue
        if values[n[0]][n[1]]<v:
            continue
        values[n[0]][n[1]] = v
        visited = path + [n]
        for d in directions:
            u = (n[0]+d[0], n[1]+d[1])
            if u in visited:
                continue
            if not valid(u) or u in points:
                continue
            q.append((u, v+1, visited))
    return result

def calc(i,j,ti,tj):
    return abs(i-ti)+abs(j-tj)

def path(details, dst):
    p = []
    i = dst[0]
    j = dst[1]
    while not (details[i][j][0]==i and details[i][j][1]==j):
        p.append((i,j))
        i,j = details[i][j][0] , details[i][j][1]
    p.append((i,j))
    p.reverse()
    return p

def astar(src, dst, points):
    closed = [[False for _ in range(size)] for _ in range(size)]
    # parent i, parent j, (g+h), g (dist src to this), h (dist to dst using heuristic)
    details =[[(0,0,math.inf,math.inf, 0) for _ in range(size)] for _ in range(size)]

    i,j=src
    details[i][j]=(i,j,0,0,0)
    olist = []
    heapq.heappush(olist, (0.0, i,j))

    found = False

    while len(olist)>0:
        _,i,j = heapq.heappop(olist)
        closed[i][j]=True
        for d in directions:
            ni = i+d[0]
            nj = j+d[1]

            if not valid((ni,nj)) or (ni,nj) in points or closed[ni][nj]:
                continue
            g = details[i][j][3] +1
            if (ni,nj)==dst:
                details[ni][nj]=(i,j,g,g,0)
                return True,g, path(details, dst)
            else:
                h = calc(i,j,*dst)
                f = g+h
                cf=details[ni][nj][2]
                if cf==math.inf or cf>f:
                    heapq.heappush(olist, (f, ni,nj))
                    details[ni][nj]=(i,j,f,g,h)
    return False,0,[]

for line in lines:
    x,y = line.split(',')
    points.append((int(x),int(y)))

p = []
f = False
g = 0
for i in range(limit, len(points)):
    c = points[:i+1]
    if points[i] in p or len(p)==0:
        f,g,p=astar((0,0), (size-1, size-1), c)
    if not f:
        print(points[i][1],",",points[i][0])
        break
    #print(i,"/",len(points)," ",g)

44 , 20


### --- Day 17: Chronospatial Computer ---

The Historians push the button on their strange device, but this time, you all just feel like you're falling.

"Situation critical", the device announces in a familiar voice. "Bootstrapping process failed. Initializing debugger...."

The small handheld device suddenly unfolds into an entire computer! The Historians look around nervously before one of them tosses it to you.

This seems to be a 3-bit computer: its program is a list of 3-bit numbers (0 through 7), like 0,1,2,3. The computer also has three registers named A, B, and C, but these registers aren't limited to 3 bits and can instead hold any integer.

The computer knows eight instructions, each identified by a 3-bit number (called the instruction's opcode). Each instruction also reads the 3-bit number after it as an input; this is called its operand.

A number called the instruction pointer identifies the position in the program from which the next opcode will be read; it starts at 0, pointing at the first 3-bit number in the program. Except for jump instructions, the instruction pointer increases by 2 after each instruction is processed (to move past the instruction's opcode and its operand). If the computer tries to read an opcode past the end of the program, it instead halts.

So, the program 0,1,2,3 would run the instruction whose opcode is 0 and pass it the operand 1, then run the instruction having opcode 2 and pass it the operand 3, then halt.

There are two types of operands; each instruction specifies the type of its operand. The value of a literal operand is the operand itself. For example, the value of the literal operand 7 is the number 7. The value of a combo operand can be found as follows:

Combo operands 0 through 3 represent literal values 0 through 3.
Combo operand 4 represents the value of register A.
Combo operand 5 represents the value of register B.
Combo operand 6 represents the value of register C.
Combo operand 7 is reserved and will not appear in valid programs.
The eight instructions are as follows:

The adv instruction (opcode 0) performs division. The numerator is the value in the A register. The denominator is found by raising 2 to the power of the instruction's combo operand. (So, an operand of 2 would divide A by 4 (2^2); an operand of 5 would divide A by 2^B.) The result of the division operation is truncated to an integer and then written to the A register.

The bxl instruction (opcode 1) calculates the bitwise XOR of register B and the instruction's literal operand, then stores the result in register B.

The bst instruction (opcode 2) calculates the value of its combo operand modulo 8 (thereby keeping only its lowest 3 bits), then writes that value to the B register.

The jnz instruction (opcode 3) does nothing if the A register is 0. However, if the A register is not zero, it jumps by setting the instruction pointer to the value of its literal operand; if this instruction jumps, the instruction pointer is not increased by 2 after this instruction.

The bxc instruction (opcode 4) calculates the bitwise XOR of register B and register C, then stores the result in register B. (For legacy reasons, this instruction reads an operand but ignores it.)

The out instruction (opcode 5) calculates the value of its combo operand modulo 8, then outputs that value. (If a program outputs multiple values, they are separated by commas.)

The bdv instruction (opcode 6) works exactly like the adv instruction except that the result is stored in the B register. (The numerator is still read from the A register.)

The cdv instruction (opcode 7) works exactly like the adv instruction except that the result is stored in the C register. (The numerator is still read from the A register.)

Here are some examples of instruction operation:

If register C contains 9, the program 2,6 would set register B to 1.
If register A contains 10, the program 5,0,5,1,5,4 would output 0,1,2.
If register A contains 2024, the program 0,1,5,4,3,0 would output 4,2,5,6,7,7,7,7,3,1,0 and leave 0 in register A.
If register B contains 29, the program 1,7 would set register B to 26.
If register B contains 2024 and register C contains 43690, the program 4,0 would set register B to 44354.
The Historians' strange device has finished initializing its debugger and is displaying some information about the program it is trying to run (your puzzle input). For example:

Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
Your first task is to determine what the program is trying to output. To do this, initialize the registers to the given values, then run the given program, collecting any output produced by out instructions. (Always join the values produced by out instructions with commas.) After the above program halts, its final output will be 4,6,3,5,6,3,5,2,1,0.

Using the information provided by the debugger, initialize the registers to the given values, then run the program. Once it halts, what do you get if you use commas to join the values it output into a single string?

In [9]:
# Parse the input file
with open('input/day_17.txt') as f:
    lines = f.read().splitlines()



def combo(operand,a,b,c):
    if operand<4:
        return operand
    return [a,b,c][operand-4]

def simulate(code,a,b,c,output, pc=0):

    while pc >=0 and pc<len(code):
        command = code[pc]
        operand = code[pc+1]
        #print(pc,a,b,c, command, operand)

        if command == 0:#adv
            a = a// (2**combo(operand, a,b,c))
        if command == 1: #bxl
            b = b ^ operand
        if command ==2: #bst
            b = combo(operand,a,b,c)%8
        if command == 3 and a!=0:#jnz
            pc = operand
        else:
            pc+=2
        if command ==4: #bxc
            b = b^c
        if command ==5: #out
            output.append(combo(operand,a,b,c)%8)
        if command ==6: #bdv
            b = a// (2**combo(operand, a,b,c))
        if command ==7: #cdv
            c = a// (2**combo(operand, a,b,c))

        #print(pc,a,b,c, command, operand)
        #print()


print(lines)

flag = True

operands = []
program = []

for line in lines:
    if line == "":
        flag = False

    if flag:
        a = int(line.split(":")[1])
        operands.append(int(a))
    elif not flag and line != "":
        program = [int(x) for x in line.split(':')[1].split(",")]

print(operands)
print(program)

output = []

simulate(program, operands[0], operands[1], operands[2], output)
print(",".join([str(x) for x in output]))


['Register A: 59590048', 'Register B: 0', 'Register C: 0', '', 'Program: 2,4,1,5,7,5,0,3,1,6,4,3,5,5,3,0']
[59590048, 0, 0]
[2, 4, 1, 5, 7, 5, 0, 3, 1, 6, 4, 3, 5, 5, 3, 0]
6,5,7,4,5,7,3,1,0


### --- Part Two ---

Digging deeper in the device's manual, you discover the problem: this program is supposed to output another copy of the program! Unfortunately, the value in register A seems to have been corrupted. You'll need to find a new value to which you can initialize register A so that the program's output instructions produce an exact copy of the program itself.

For example:

Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0
This program outputs a copy of itself if register A is instead initialized to 117440. (The original initial value of register A, 2024, is ignored.)

What is the lowest positive initial value for register A that causes the program to output a copy of itself?

In [18]:
from collections import deque

# Parse the input file
with open('input/day_17.txt') as f:
    lines = f.read().splitlines()



def combo(operand,a,b,c):
    if operand<4:
        return operand
    return [a,b,c][operand-4]

def simulate(code,a,b,c,output, pc=0):

    while pc >=0 and pc<len(code):
        command = code[pc]
        operand = code[pc+1]
        #print(pc,a,b,c, command, operand)

        if command == 0:#adv
            a = a// (2**combo(operand, a,b,c))
        if command == 1: #bxl
            b = b ^ operand
        if command ==2: #bst
            b = combo(operand,a,b,c)%8
        if command == 3 and a!=0:#jnz
            pc = operand
        else:
            pc+=2
        if command ==4: #bxc
            b = b^c
        if command ==5: #out
            output.append(combo(operand,a,b,c)%8)
        if command ==6: #bdv
            b = a// (2**combo(operand, a,b,c))
        if command ==7: #cdv
            c = a// (2**combo(operand, a,b,c))

        #print(pc,a,b,c, command, operand)
        #print()

flag = True

operands = []
program = []

for line in lines:
    if line == "":
        flag = False

    if flag:
        a = int(line.split(":")[1])
        operands.append(int(a))
    elif not flag and line != "":
        program = [int(x) for x in line.split(':')[1].split(",")]

print(operands)
print(program)

output = []

q = deque([0])
i = 0
while q:
    expected = program[len(program)-i-1:]
    if i==len(program):
        print(min(q))
        exit()
    for _ in range(len(q)):
        v = q.popleft()
        for k in range(8):
            a = 8*v + k
            output=[]
            simulate(program, operands[0], operands[1], operands[2], output)
            if output == expected:
                q.append(a)
    i+=1

# print deque value
print(list(q))

[2024, 0, 0]
[0, 3, 5, 4, 3, 0]
[]


### --- Day 16: Reindeer Maze ---

It's time again for the Reindeer Olympics! This year, the big event is the Reindeer Maze, where the Reindeer compete for the lowest score.

You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?

The Reindeer start on the Start Tile (marked S) facing East and need to reach the End Tile (marked E). They can move forward one tile at a time (increasing their score by 1 point), but never into a wall (#). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by 1000 points).

To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:

```
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
```
There are many paths through this maze, but taking any of the best paths would incur a score of only 7036. This can be achieved by taking a total of 36 steps forward and turning 90 degrees a total of 7 times:

```
###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#..>>>>>>>>v#^#
###^#.#####v#^#
#>>^#.....#v#^#
#^#.#.###.#v#^#
#^....#...#v#^#
#^###.#.#.#v#^#
#S..#.....#>>^#
###############
```

Here's a second example:

```
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
```

In this maze, the best paths cost 11048 points; following one such path would look like this:

```
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#^#
#.#.#.#...#...#^#
#.#.#.#.###.#.#^#
#>>v#.#.#.....#^#
#^#v#.#.#.#####^#
#^#v..#.#.#>>>>^#
#^#v#####.#^###.#
#^#v#..>>>>^#...#
#^#v###^#####.###
#^#v#>>^#.....#.#
#^#v#^#####.###.#
#^#v#^........#.#
#^#v#^#########.#
#S#>>^..........#
#################
```

Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.

Analyze your map carefully. What is the lowest score a Reindeer could possibly get?

In [29]:
import time

# Parse the input file
with open('input/day_16.txt') as f:
    lines = f.read().splitlines()

map = [list(line.strip()) for line in lines if line.strip()]


def beautiful_print(_map):
    for row in _map:
        for column in row:
            print(column, end="")
        print()


directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]


def turn_right(cur_dir):
    return directions[(directions.index(cur_dir) + 1 + len(directions)) % len(directions)]


def turn_left(cur_dir):
    return directions[(directions.index(cur_dir) - 1 + len(directions)) % len(directions)]


def bfs(_map):
    queue = deque()
    start = (len(_map) - 2, 1, (0, 1), 0)  # x, y, direction, score
    _map[len(_map) - 2][1] = 0
    queue.append(start)

    while queue:
        current = queue.popleft()
        cur_x = current[0]
        cur_y = current[1]
        cur_dir = current[2]
        cur_score = current[3]

        allowed_directions_n_score = [
            (cur_dir, cur_score + 1),
            (turn_left(cur_dir), cur_score + 1001),
            (turn_right(cur_dir), cur_score + 1001)
        ]

        for new_dir, new_score in allowed_directions_n_score:
            new_x, new_y = cur_x + new_dir[0], cur_y + new_dir[1]
            if _map[new_x][new_y] == "#":
                continue

            if _map[new_x][new_y] in [".", "E"] or (isinstance(_map[new_x][new_y], int) and _map[new_x][new_y] > new_score):
                _map[new_x][new_y] = new_score
                queue.append((new_x, new_y, new_dir, new_score))

    return _map[1][len(_map[1]) - 2]


def resolve():
    my_map = map
    result = bfs(my_map)
    return result

start_time = time.time()
result = resolve()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Result: {result}")
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Result: 122492
Elapsed time: 0.009780 seconds


### --- Part Two ---

Now that you know what the best paths look like, you can figure out the best spot to sit.

Every non-wall tile (S, ., or E) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is whether the tile is on one of the best paths through the maze. If you sit somewhere else, you'd miss all the action!

So, you'll need to determine which tiles are part of any best path through the maze, including the S and E tiles.

In the first example, there are 45 tiles (marked O) that are part of at least one of the various best paths through the maze:

```
###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
```

In the second example, there are 64 tiles that are part of at least one of the best paths:

```
#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################
```

Analyze your map further. How many tiles are part of at least one of the best paths through the maze?

In [30]:
import time

# Parse the input file
with open('input/day_16.txt') as f:
    lines = f.read().splitlines()

map = [list(line.strip()) for line in lines if line.strip()]


def beautiful_print(_map):
    for row in _map:
        for column in row:
            print(column, end="")
        print()


directions = [(1, 0), (0, -1), (-1, 0), (0, 1)]


def turn_right(cur_dir):
    return directions[(directions.index(cur_dir) + 1 + len(directions)) % len(directions)]


def turn_left(cur_dir):
    return directions[(directions.index(cur_dir) - 1 + len(directions)) % len(directions)]


def backwards_bfs(_map):
    queue = deque()
    res = 1
    start_1 = (1, len(_map[1]) - 2, (1, 0), _map[1][len(_map[1]) - 2])  # x, y, direction, score
    start_2 = (1, len(_map[1]) - 2, (0, -1), _map[1][len(_map[1]) - 2])  # x, y, direction, score
    queue.append(start_1)
    queue.append(start_2)
    visited = set()

    while queue:
        current = queue.popleft()
        cur_x = current[0]
        cur_y = current[1]
        cur_dir = current[2]
        cur_score = current[3]

        allowed_directions_n_score = [
            (cur_dir, cur_score - 1),
            (turn_left(cur_dir), cur_score - 1001),
            (turn_right(cur_dir), cur_score - 1001)
        ]

        for new_dir, new_score in allowed_directions_n_score:
            new_x, new_y = cur_x + new_dir[0], cur_y + new_dir[1]
            if isinstance(_map[new_x][new_y], int) and (_map[new_x][new_y] in [new_score, new_score - 1000]) and (new_x, new_y) not in visited:
                res += 1
                queue.append((new_x, new_y, new_dir, new_score))
                visited.add((new_x, new_y))

    return res

def bfs(_map):
    queue = deque()
    start = (len(_map) - 2, 1, (0, 1), 0)  # x, y, direction, score
    _map[len(_map) - 2][1] = 0
    queue.append(start)

    while queue:
        current = queue.popleft()
        cur_x = current[0]
        cur_y = current[1]
        cur_dir = current[2]
        cur_score = current[3]

        allowed_directions_n_score = [
            (cur_dir, cur_score + 1),
            (turn_left(cur_dir), cur_score + 1001),
            (turn_right(cur_dir), cur_score + 1001)
        ]

        for new_dir, new_score in allowed_directions_n_score:
            new_x, new_y = cur_x + new_dir[0], cur_y + new_dir[1]
            if _map[new_x][new_y] == "#":
                continue

            if _map[new_x][new_y] in [".", "E"] or (isinstance(_map[new_x][new_y], int) and _map[new_x][new_y] > new_score):
                _map[new_x][new_y] = new_score
                queue.append((new_x, new_y, new_dir, new_score))

    return _map[1][len(_map[1]) - 2]


def resolve():
    my_map = map
    bfs(my_map)
    return backwards_bfs(my_map)


start_time = time.time()
result = resolve()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Result: {result}")
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Result: 520
Elapsed time: 0.010736 seconds


### --- Day 15: Warehouse Woes ---

You appear back inside your own mini submarine! Each Historian drives their mini submarine in a different direction; maybe the Chief has his own submarine down here somewhere as well?

You look up to see a vast school of lanternfish swimming past you. On closer inspection, they seem quite anxious, so you drive your mini submarine over to see if you can help.

Because lanternfish populations grow rapidly, they need a lot of food, and that food needs to be stored somewhere. That's why these lanternfish have built elaborate warehouse complexes operated by robots!

These lanternfish seem so anxious because they have lost control of the robot that operates one of their most important warehouses! It is currently running amok, pushing around boxes in the warehouse with no regard for lanternfish logistics or lanternfish inventory management strategies.

Right now, none of the lanternfish are brave enough to swim up to an unpredictable robot so they could shut it off. However, if you could anticipate the robot's movements, maybe they could find a safe option.

The lanternfish already have a map of the warehouse and a list of movements the robot will attempt to make (your puzzle input). The problem is that the movements will sometimes fail as boxes are shifted around, making the actual movements of the robot difficult to predict.

For example:

```

##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
```

As the robot (@) attempts to move, if there are any boxes (O) in the way, the robot will also attempt to push those boxes. However, if this action would cause the robot or a box to move into a wall (#), nothing moves instead, including the robot. The initial positions of these are shown on the map at the top of the document the lanternfish gave you.

The rest of the document describes the moves (^ for up, v for down, < for left, > for right) that the robot will attempt to make, in order. (The moves form a single giant sequence; they are broken into multiple lines just to make copy-pasting easier. Newlines within the move sequence should be ignored.)

Here is a smaller example to get started:

########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<
The larger example has many more moves; after the robot has finished those moves, the warehouse would look like this:

##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########
The lanternfish use their own custom Goods Positioning System (GPS for short) to track the locations of the boxes. The GPS coordinate of a box is equal to 100 times its distance from the top edge of the map plus its distance from the left edge of the map. (This process does not stop at wall tiles; measure all the way to the edges of the map.)

So, the box shown below has a distance of 1 from the top edge of the map and 4 from the left edge of the map, resulting in a GPS coordinate of 100 * 1 + 4 = 104.

#######
#...O..
#......

The lanternfish would like to know the sum of all boxes' GPS coordinates after the robot finishes moving. In the larger example, the sum of all boxes' GPS coordinates is 10092. In the smaller example, the sum is 2028.

Predict the motion of the robot and boxes in the warehouse. After the robot is finished moving, what is the sum of all boxes' GPS coordinates?

In [17]:
# Parse the input file
with open('input/day_15.txt') as f:
    lines = f.read().splitlines()

d={
    "<":(0,-1),
    "^":(-1,0),
    ">":(0,1),
    "v":(1,0)
}

def printa(c,mat):
    if 1>0:
        return
    print(c)
    for l in mat:
        print("".join(l))

def apply(mat,p, c):
    move=False
    delta = d[c]
    nx = p[0]+delta[0]
    ny = p[1]+delta[1]

    bx = nx
    by = ny
    box = False

    while mat[bx][by] == 'O':
        box=True
        bx = bx+delta[0]
        by = by+delta[1]
    if box and mat[bx][by]=='.':
        mat[bx][by]='O'
        move = True
    if mat[nx][ny]=='.':
        move=True

    if move:
        mat[p[0]][p[1]] = '.'
        mat[nx][ny]='@'
        return (nx,ny)
    return p

mat = []
commands = ''

flag = False
for line in lines:
    if line == '':
        flag = True
        continue
    if flag:
        commands = ''.join([commands, line])
    else:
        mat.append(list(line))


s = (0,0)
for i in range(len(mat)):
    for j in range(len(mat[i])):
        if mat[i][j]=='@':
            s = (i,j)
for c in commands:
    s=apply(mat,s,c)
    printa(c,mat)
result = 0
for i in range(len(mat)):
    for j in range(len(mat[i])):
        if mat[i][j]=='O':
            result+= 100*i+j
print(result)


1436690


### --- Part Two ---

The lanternfish use your information to find a safe moment to swim in and turn off the malfunctioning robot! Just as they start preparing a festival in your honor, reports start coming in that a second warehouse's robot is also malfunctioning.

This warehouse's layout is surprisingly similar to the one you just helped. There is one key difference: everything except the robot is twice as wide! The robot's list of movements doesn't change.

To get the wider warehouse's map, start with your original map and, for each tile, make the following changes:

If the tile is #, the new map contains ## instead.
If the tile is O, the new map contains [] instead.
If the tile is ., the new map contains .. instead.
If the tile is @, the new map contains @. instead.
This will produce a new warehouse map which is twice as wide and with wide boxes that are represented by []. (The robot does not change size.)

The larger example from before would now look like this:

```
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
```
Because boxes are now twice as wide but the robot is still the same size and speed, boxes can be aligned such that they directly push two other boxes at once. For example, consider this situation:

#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^
After appropriately resizing this map, the robot would push around these boxes as follows:


This warehouse also uses GPS to locate the boxes. For these larger boxes, distances are measured from the edge of the map to the closest edge of the box in question. So, the box shown below has a distance of 1 from the top edge of the map and 5 from the left edge of the map, resulting in a GPS coordinate of 100 * 1 + 5 = 105.

##########
##...[]...
##........
In the scaled-up version of the larger example from above, after the robot has finished all of its moves, the warehouse would look like this:

```
####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################
```

The sum of these boxes' GPS coordinates is 9021.

Predict the motion of the robot and boxes in this new, scaled-up warehouse. What is the sum of all boxes' final GPS coordinates?

In [19]:
# Parse the input file
with open('input/day_15.txt') as f:
    lines = f.read().splitlines()

import sys

d={
    "<":(0,-1),
    "^":(-1,0),
    ">":(0,1),
    "v":(1,0)
}

def printa(c,mat):
    if 1>0:
        return
    print(c)
    for l in mat:
        print("".join(l))
    print()

def check(mat, delta, opts):
    nopts = set()
    for ox,oy in opts:
        v = mat[ox+delta][oy]
        if v == '.':
            continue
        if v =='#':
            return 0
        nopts.add((ox+delta,oy))
        nopts.add((ox+delta,oy + (1 if v=='[' else -1)))
    if len(nopts) == 0:
        return 1
    r = check(mat, delta, nopts)
    if r == 0:
        return 0
    return r+1

def edit(mat, delta, opts, fit):
    if fit<0:
        return
    nopts = set()
    maybeDelete = set()
    for ox,oy,nv in opts:
        v = mat[ox+delta][oy]
        if v in '[]':
            nopts.add((ox+delta, oy, v))
            ny = oy +(1 if v=='[' else -1)
            nopts.add((ox+delta, ny, '['if v==']'else']'))
            maybeDelete.add((ox, ny))
        mat[ox+delta][oy]=nv
    for x,y,_ in opts:
        maybeDelete.discard((x,y))
    for x, y in maybeDelete:
        mat[x+delta][y]='.'
    edit(mat, delta, nopts, fit-1)

def moveh(mat,p,c):
    delta = d[c]
    nx = p[0]+delta[0]
    ny = p[1]+delta[1]

    bx,by = nx,ny
    while mat[bx][by] in "[]":
        bx = bx+delta[0]
        by = by+delta[1]
    if mat[bx][by]=='#':
        return p

    mat[p[0]][p[1]]='.'
    mat[nx][ny] = '@'
    box ="[]"
    b=0 if c=='>' else 1
    for i in range(abs(ny-by)):
        mat[bx][ny+((i+1)*delta[1])]=box[(b+i)%2]
        i+=1
    return (nx,ny)

def movev(mat,p,c):
    delta = d[c]
    nx = p[0]+delta[0]
    ny = p[1]+delta[1]

    v = mat[nx][ny]

    if v=='.':
        mat[p[0]][p[1]]='.'
        mat[nx][ny] = '@'
        return (nx,ny)
    if v=='#':
        return p

    #found box, check if fit
    fit = check(mat, delta[0], [(nx,ny), (nx,ny + (1 if v=='[' else -1))])

    if fit == 0 :
        return p

    #fit has the n# of rows will be edited
    mat[p[0]][p[1]]='.'
    edit(mat, delta[0], [(*p, '@')], fit)
    return (nx,ny)

def apply(mat,p, c):
    if c in "<>":
        return moveh(mat, p,c)
    else:
        return movev(mat,p,c)

mat = []
commands = ''

flag = False
for line in lines:
    if line == '':
        flag = True
        continue
    if flag:
        commands = ''.join([commands, line])
    else:
        l = list(line)
        x=[]
        for c in l:
            if c=='@':
                x.append('@')
                x.append('.')
            elif c=='O':
                x.append('[')
                x.append(']')
            else:
                x.append(c)
                x.append(c)
        mat.append(x)


s = (0,0)
for i in range(len(mat)):
    for j in range(len(mat[i])):
        if mat[i][j]=='@':
            s = (i,j)
for c in commands:
    s=apply(mat,s,c)
    printa(c,mat)
result = 0
for i in range(len(mat)):
    for j in range(len(mat[i])):
        if mat[i][j]=='[':
            result+= 100*i+j
print(result)


1482350


### --- Day 14: Restroom Redoubt ---

One of The Historians needs to use the bathroom; fortunately, you know there's a bathroom near an unvisited location on their list, and so you're all quickly teleported directly to the lobby of Easter Bunny Headquarters.

Unfortunately, EBHQ seems to have "improved" bathroom security again after your last visit. The area outside the bathroom is swarming with robots!

To get The Historian safely to the bathroom, you'll need a way to predict where the robots will be in the future. Fortunately, they all seem to be moving on the tile floor in predictable straight lines.

You make a list (your puzzle input) of all of the robots' current positions (p) and velocities (v), one robot per line. For example:

```
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
```

Each robot's position is given as p=x,y where x represents the number of tiles the robot is from the left wall and y represents the number of tiles from the top wall (when viewed from above). So, a position of p=0,0 means the robot is all the way in the top-left corner.

Each robot's velocity is given as v=x,y where x and y are given in tiles per second. Positive x means the robot is moving to the right, and positive y means the robot is moving down. So, a velocity of v=1,-2 means that each second, the robot moves 1 tile to the right and 2 tiles up.

The robots outside the actual bathroom are in a space which is 101 tiles wide and 103 tiles tall (when viewed from above). However, in this example, the robots are in a space which is only 11 tiles wide and 7 tiles tall.

The robots are good at navigating over/under each other (due to a combination of springs, extendable legs, and quadcopters), so they can share the same tile and don't interact with each other. Visually, the number of robots on each tile in this example looks like this:

```
1.12.......
...........
...........
......11.11
1.1........
.........1.
.......1...
```

The Historian can't wait much longer, so you don't have to simulate the robots for very long. Where will the robots be after 100 seconds?

In the above example, the number of robots on each tile after 100 seconds has elapsed looks like this:

```
......2..1.
...........
1..........
.11........
.....1.....
...12......
.1....1....
```

To determine the safest area, count the number of robots in each quadrant after 100 seconds. Robots that are exactly in the middle (horizontally or vertically) don't count as being in any quadrant, so the only relevant robots are:
These robots have a unique feature for maximum bathroom security: they can teleport. When a robot would run into an edge of the space they're in, they instead teleport to the other side, effectively wrapping around the edges. Here is what robot p=2,4 v=2,-3 does for the first few seconds:

..... 2..1.
..... .....
1.... .....

..... .....
...12 .....
.1... 1....
In this example, the quadrants contain 1, 3, 4, and 1 robot. Multiplying these together gives a total safety factor of 12.

Predict the motion of the robots in your list within a space which is 101 tiles wide and 103 tiles tall. What will the safety factor be after exactly 100 seconds have elapsed?

In [11]:
# Parse the input file
with open('input/day_14.txt') as f:
    lines = f.read().splitlines()

def parse(x):
    p,v = x.strip().split()
    px,py = p.split("=")[1].split(",")
    vx,vy = v.split("=")[1].split(",")
    return (int(px), int(py)), (int(vx), int(vy))

r = [parse(x) for x in lines]

rounds = 100
size=(101,103)
mx = size[0]//2
my = size[1]//2

tl = 0
tr = 0
bl = 0
br = 0

for p,v in r:
    nx = (p[0] +v[0]*rounds)%size[0]
    ny = (p[1] +v[1]*rounds)%size[1]
    if nx<mx and ny<my:
            tl+=1
    if nx<mx and ny>my:
            tr+=1
    if nx>mx and ny<my:
            bl+=1
    if nx>mx and ny>my:
            br+=1
print(tl,tr,bl,br)
print(tl*tr*bl*br)


116 126 123 130
233709840


### --- Part Two ---
During the bathroom break, someone notices that these robots seem awfully similar to ones built and used at the North Pole. If they're the same type of robots, they should have a hard-coded Easter egg: very rarely, most of the robots should arrange themselves into a picture of a Christmas tree.

What is the fewest number of seconds that must elapse for the robots to display the Easter egg?

In [12]:
import math

# Parse the input file
with open('input/day_14.txt') as f:
    lines = f.read().splitlines()

def parse(x):
    p,v = x.strip().split()
    px,py = p.split("=")[1].split(",")
    vx,vy = v.split("=")[1].split(",")
    return (int(px), int(py)), (int(vx), int(vy))

r = [parse(x) for x in lines]

rounds = 100
size=(101,103)
mx = size[0]//2
my = size[1]//2

sfmin=math.inf
sfmax=0
resp = 0

for i in range(size[0]*size[1]):
    tl = 0
    tr = 0
    bl = 0
    br = 0
    for p,v in r:
        nx = (p[0] +v[0]*i)%size[0]
        ny = (p[1] +v[1]*i)%size[1]
        if nx<mx and ny<my:
                tl+=1
        if nx<mx and ny>my:
                tr+=1
        if nx>mx and ny<my:
                bl+=1
        if nx>mx and ny>my:
                br+=1
    sf = tl*tr*bl*br
    #print(i,sf)
    sfmax=max(sfmax,sf)
    if sf<sfmin:
        print(i, sf)
        sfmin=sf
        resp = i
print(resp)


0 224519680
3 221848704
4 208156704
28 67882880
234 66214590
1367 65699850
1779 63697313
4766 62853120
6620 23979330
6620


### --- Day 13: Claw Contraption : Part 1 ---

Next up: the lobby of a resort on a tropical island. The Historians take a moment to admire the hexagonal floor tiles before spreading out.

Fortunately, it looks like the resort has a new arcade! Maybe you can win some prizes from the claw machines?

The claw machines here are a little unusual. Instead of a joystick or directional buttons to control the claw, these machines have two buttons labeled A and B. Worse, you can't just put in a token and play; it costs 3 tokens to push the A button and 1 token to push the B button.

With a little experimentation, you figure out that each machine's buttons are configured to move the claw a specific amount to the right (along the X axis) and a specific amount forward (along the Y axis) each time that button is pressed.

Each machine contains one prize; to win the prize, the claw must be positioned exactly above the prize on both the X and Y axes.

You wonder: what is the smallest number of tokens you would have to spend to win as many prizes as possible? You assemble a list of every machine's button behavior and prize location (your puzzle input). For example:

```
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279
```

This list describes the button configuration and prize location of four different claw machines.

For now, consider just the first claw machine in the list:

Pushing the machine's A button would move the claw 94 units along the X axis and 34 units along the Y axis.
Pushing the B button would move the claw 22 units along the X axis and 67 units along the Y axis.
The prize is located at X=8400, Y=5400; this means that from the claw's initial position, it would need to move exactly 8400 units along the X axis and exactly 5400 units along the Y axis to be perfectly aligned with the prize in this machine.
The cheapest way to win the prize is by pushing the A button 80 times and the B button 40 times. This would line up the claw along the X axis (because 80*94 + 40*22 = 8400) and along the Y axis (because 80*34 + 40*67 = 5400). Doing this would cost 80*3 tokens for the A presses and 40*1 for the B presses, a total of 280 tokens.

For the second and fourth claw machines, there is no combination of A and B presses that will ever win a prize.

For the third claw machine, the cheapest way to win the prize is by pushing the A button 38 times and the B button 86 times. Doing this would cost a total of 200 tokens.

So, the most prizes you could possibly win is two; the minimum tokens you would have to spend to win all (two) prizes is 480.

You estimate that each button would need to be pressed no more than 100 times to win a prize. How else would someone be expected to play?

Figure out how to win as many prizes as possible. What is the fewest tokens you would have to spend to win all possible prizes?



In [4]:
# Parse the input file
with open('input/day_13.txt') as f:
    lines = f.read().splitlines()

#print(lines)

def parseLine(line, div='+'):
    x,y = line.split(':')[1].strip().split(",")
    x = int(x.split(div)[1].strip())
    y = int(y.split(div)[1].strip())
    return x,y

def solve(la, lb, lp):

    # Get the x, y from the input
    xa,ya = parseLine(la)
    xb,yb = parseLine(lb)
    xp,yp = parseLine(lp, "=")

    # I: xa*A + xb*B = xp
    # II: ya*A + yb*B = yp II

    # III (from I): A = (xp - xb*B)/xa
    # IV (apply III on II): ya*[(xp - xb*B)/xa] + yb*B = yp
    #    => ya*(xp-xb*B) + xa*yb*B = yp*xa
    #    => B*(xa*yb -ya*xb) = yp*xa - ya*xp
    #    => B = (yp*xa - ya*xp)/(xa*yb -ya*xb)
    # invalid if B is not int
    # apply IV on III
    # invalid if A is not int

    b =  (yp*xa - ya*xp)/(xa*yb -ya*xb)
    a = (xp - xb*b)/xa
    if int(b)!=b or int(a)!=a:
        return 0
    if a>100 or a<0 or b>100 or b<0:
        return 0
    return a*3 + b

tokens = 0
prob = []

for line in lines:
    if len(line.strip()) == 0:
        tokens+=solve(*prob)
        prob = []
        continue
    prob.append(line.strip())
tokens+=solve(*prob)
print(tokens)

39996.0


### --- Part Two ---

As you go to win the first prize, you discover that the claw is nowhere near where you expected it would be. Due to a unit conversion error in your measurements, the position of every prize is actually 10000000000000 higher on both the X and Y axis!

Add 10000000000000 to the X and Y position of every prize. After making this change, the example above would now look like this:

```
Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=10000000008400, Y=10000000005400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=10000000012748, Y=10000000012176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=10000000007870, Y=10000000006450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=10000000018641, Y=10000000010279
```

Now, it is only possible to win a prize on the second and fourth claw machines. Unfortunately, it will take many more than 100 presses to do so.

Using the corrected prize coordinates, figure out how to win as many prizes as possible. What is the fewest tokens you would have to spend to win all possible prizes?

In [6]:
# Parse the input file
with open('input/day_13.txt') as f:
    lines = f.read().splitlines()

#print(lines)

def parseLine(line, div='+'):
    x,y = line.split(':')[1].strip().split(",")
    x = int(x.split(div)[1].strip())
    y = int(y.split(div)[1].strip())
    return x,y

def solve(la, lb, lp):

    # Get the x, y from the input
    xa,ya = parseLine(la)
    xb,yb = parseLine(lb)
    xp,yp = parseLine(lp, "=")
    xp+=10000000000000
    yp+=10000000000000

    # I: xa*A + xb*B = xp
    # II: ya*A + yb*B = yp II

    # III (from I): A = (xp - xb*B)/xa
    # IV (apply III on II): ya*[(xp - xb*B)/xa] + yb*B = yp
    #    => ya*(xp-xb*B) + xa*yb*B = yp*xa
    #    => B*(xa*yb -ya*xb) = yp*xa - ya*xp
    #    => B = (yp*xa - ya*xp)/(xa*yb -ya*xb)
    # invalid if B is not int
    # apply IV on III
    # invalid if A is not int

    b =  (yp*xa - ya*xp)/(xa*yb -ya*xb)
    a = (xp - xb*b)/xa
    if int(b)-b or int(a)!=a:
        return 0
    if  a<0 or b<0:
        return 0
    return a*3 + b

tokens = 0
prob = []

for line in lines:
    if len(line.strip()) == 0:
        tokens+=solve(*prob)
        prob = []
        continue
    prob.append(line.strip())
tokens+=solve(*prob)
print(tokens)

73267584326867.0


### --- Day 12: Garden Groups ---

Why not search for the Chief Historian near the gardener and his massive farm? There's plenty of food, so The Historians grab something to eat while they search.

You're about to settle near a complex arrangement of garden plots when some Elves ask if you can lend a hand. They'd like to set up fences around each region of garden plots, but they can't figure out how much fence they need to order or how much it will cost. They hand you a map (your puzzle input) of the garden plots.

Each garden plot grows only a single type of plant and is indicated by a single letter on your map. When multiple garden plots are growing the same type of plant and are touching (horizontally or vertically), they form a region. For example:

```
AAAA
BBCD
BBCC
EEEC
```


This 4x4 arrangement includes garden plots growing five different types of plants (labeled A, B, C, D, and E), each grouped into their own region.

In order to accurately calculate the cost of the fence around a single region, you need to know that region's area and perimeter.

The area of a region is simply the number of garden plots the region contains. The above map's type A, B, and C plants are each in a region of area 4. The type E plants are in a region of area 3; the type D plants are in a region of area 1.

Each garden plot is a square and so has four sides. The perimeter of a region is the number of sides of garden plots in the region that do not touch another garden plot in the same region. The type A and C plants are each in a region with perimeter 10. The type B and E plants are each in a region with perimeter 8. The lone D plot forms its own region with perimeter 4.

Visually indicating the sides of plots in each region that contribute to the perimeter using - and |, the above map's regions' perimeters are measured as follows:

```
+-+-+-+-+
|A A A A|
+-+-+-+-+     +-+
              |D|
+-+-+   +-+   +-+
|B B|   |C|
+   +   + +-+
|B B|   |C C|
+-+-+   +-+ +
          |C|
+-+-+-+   +-+
|E E E|
+-+-+-+
```

Plants of the same type can appear in multiple separate regions, and regions can even appear within other regions. For example:

```
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
```

The above map contains five regions, one containing all of the O garden plots, and the other four each containing a single X plot.

The four X regions each have area 1 and perimeter 4. The region containing 21 type O plants is more complicated; in addition to its outer edge contributing a perimeter of 20, its boundary with each X region contributes an additional 4 to its perimeter, for a total perimeter of 36.

Due to "modern" business practices, the price of fence required for a region is found by multiplying that region's area by its perimeter. The total price of fencing all regions on a map is found by adding together the price of fence for every region on the map.

In the first example, region A has price 4 * 10 = 40, region B has price 4 * 8 = 32, region C has price 4 * 10 = 40, region D has price 1 * 4 = 4, and region E has price 3 * 8 = 24. So, the total price for the first example is 140.

In the second example, the region with all of the O plants has price 21 * 36 = 756, and each of the four smaller X regions has price 1 * 4 = 4, for a total price of 772 (756 + 4 + 4 + 4 + 4).

Here's a larger example:

```
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
```

It contains:

A region of R plants with price 12 * 18 = 216.
A region of I plants with price 4 * 8 = 32.
A region of C plants with price 14 * 28 = 392.
A region of F plants with price 10 * 18 = 180.
A region of V plants with price 13 * 20 = 260.
A region of J plants with price 11 * 20 = 220.
A region of C plants with price 1 * 4 = 4.
A region of E plants with price 13 * 18 = 234.
A region of I plants with price 14 * 22 = 308.
A region of M plants with price 5 * 12 = 60.
A region of S plants with price 3 * 8 = 24.
So, it has a total price of 1930.

What is the total price of fencing all regions on your map?

In [3]:
from collections import deque

# Parse the input file
with open('input/day_12.txt') as f:
    lines = f.read().splitlines()

alternatives = [
    (-1,0),
    (0,-1),
    (1,0),
    (0,1)
]

def invalid(x,y,mat):
    return not (0 <= x < len(mat) and 0 <= y < len(mat[x]))

def calc(mat, start, visit):
    v = mat[start[0]][start[1]]
    a = 0
    p = 0
    q = deque([start])
    while q:
        d = q.popleft()
        if d in visit:
            continue
        visit.add(d)
        a+=1
        for u in alternatives:
            x = d[0]+u[0]
            y = d[1]+u[1]
            if invalid(x,y,mat):
                p+=1
            elif mat[x][y]!=v:
                p+=1
            else:
                q.append((x,y))
    return a,p

visited = set()

resp = 0
for i in range(len(lines)):
    for j in range(len(lines[i])):
        if (i,j) in visited:
            continue
        a,p = calc(lines, (i, j), visited)
        print("region of ", lines[i][j], " area ",a, " perimeter ",p)
        resp += a*p
print(resp)

region of  D  area  186  perimeter  84
region of  S  area  32  perimeter  36
region of  Y  area  235  perimeter  118
region of  W  area  1  perimeter  4
region of  P  area  18  perimeter  28
region of  E  area  34  perimeter  38
region of  W  area  1  perimeter  4
region of  C  area  2  perimeter  6
region of  R  area  52  perimeter  54
region of  O  area  1  perimeter  4
region of  K  area  130  perimeter  78
region of  M  area  330  perimeter  214
region of  I  area  2  perimeter  6
region of  I  area  55  perimeter  76
region of  V  area  130  perimeter  80
region of  E  area  48  perimeter  54
region of  T  area  9  perimeter  18
region of  W  area  84  perimeter  62
region of  Q  area  66  perimeter  66
region of  P  area  40  perimeter  38
region of  B  area  1  perimeter  4
region of  A  area  128  perimeter  84
region of  J  area  3  perimeter  8
region of  S  area  92  perimeter  94
region of  G  area  1  perimeter  4
region of  E  area  1  perimeter  4
region of  T  area  1  

### --- Part Two ---

Fortunately, the Elves are trying to order so much fence that they qualify for a bulk discount!

Under the bulk discount, instead of using the perimeter to calculate the price, you need to use the number of sides each region has. Each straight section of fence counts as a side, regardless of how long it is.

Consider this example again:

```
AAAA
BBCD
BBCC
EEEC
```

The region containing type A plants has 4 sides, as does each of the regions containing plants of type B, D, and E. However, the more complex region containing the plants of type C has 8 sides!

Using the new method of calculating the per-region price by multiplying the region's area by its number of sides, regions A through E have prices 16, 16, 32, 4, and 12, respectively, for a total price of 80.

The second example above (full of type X and O plants) would have a total price of 436.

Here's a map that includes an E-shaped region full of type E plants:

```
EEEEE
EXXXX
EEEEE
EXXXX
EEEEE
```

The E-shaped region has an area of 17 and 12 sides for a price of 204. Including the two regions full of type X plants, this map has a total price of 236.

This map has a total price of 368:

```
AAAAAA
AAABBA
AAABBA
ABBAAA
ABBAAA
AAAAAA
```

It includes two regions full of type B plants (each with 4 sides) and a single region full of type A plants (with 4 sides on the outside and 8 more sides on the inside, a total of 12 sides). Be especially careful when counting the fence around regions like the one full of type A plants; in particular, each section of fence has an in-side and an out-side, so the fence does not connect across the middle of the region (where the two B regions touch diagonally). (The Elves would have used the Möbius Fencing Company instead, but their contract terms were too one-sided.)

The larger example from before now has the following updated prices:

A region of R plants with price 12 * 10 = 120.
A region of I plants with price 4 * 4 = 16.
A region of C plants with price 14 * 22 = 308.
A region of F plants with price 10 * 12 = 120.
A region of V plants with price 13 * 10 = 130.
A region of J plants with price 11 * 12 = 132.
A region of C plants with price 1 * 4 = 4.
A region of E plants with price 13 * 8 = 104.
A region of I plants with price 14 * 16 = 224.
A region of M plants with price 5 * 6 = 30.
A region of S plants with price 3 * 6 = 18.
Adding these together produces its new total price of 1206.

What is the new total price of fencing all regions on your map?

In [4]:
from collections import deque

alternatives = [
    (-1,0),
    (0,-1),
    (1,0),
    (0,1)
]
direction = {"^":((0,-1), (0,1)), ">":((1,0), (-1,0)), "v":((0,1), (0,-1)), "<":((-1,0), (1,0))}
sym = {
    (-1,0): '^',
    (0,-1): '<',
    (1,0): 'v',
    (0,1): '>'
}

def invalid(x,y,mat):
    return not (0 <= x < len(mat) and 0 <= y < len(mat[x]))


def walkBorder(e):
    s = 0
    for k in e:
        # k = ^>v<
        x = e[k]
        a,b = direction[k]
        while len(x)>0:
            p = x.pop()
            #walk into a removing
            k = (p[0]+a[0], p[1]+a[1])
            while k in x:
                x.remove(k)
                k = (k[0]+a[0], k[1]+a[1])

            #walk into b removing
            k = (p[0]+b[0], p[1]+b[1])
            while k in x:
                x.remove(k)
                k = (k[0]+b[0], k[1]+b[1])
            s+=1
    return s

# Parse the input file
with open('input/day_12.txt') as f:
    lines = f.read().splitlines()


def flood(mat, start, visit):
    v = mat[start[0]][start[1]]
    a = 0
    p = 0
    e = {}
    for k in sym.values():
        e[k]=set()
    q = deque([start])
    while q:
        d = q.popleft()
        if d in visit:
            continue
        visit.add(d)
        a+=1
        for u in alternatives:
            x = d[0]+u[0]
            y = d[1]+u[1]
            if invalid(x,y,mat):
                p+=1
                e[sym[u]].add(d)
            elif mat[x][y]!=v:
                p+=1
                e[sym[u]].add(d)
            else:
                q.append((x,y))
    return a,p,e

visited = set()

resp = 0
for i in range(len(lines)):
    for j in range(len(lines[i])):
        if (i,j) in visited:
            continue
        a,p,e = flood(lines, (i, j), visited)
        print("region of ", lines[i][j], " area ",a, " perimeter ",p)
        #print("external ",e)
        # i,j is a top-left corner
        s = walkBorder(e)
        print("slices ", s)
        resp += a*s
print(resp)

region of  D  area  186  perimeter  84
slices  38
region of  S  area  32  perimeter  36
slices  14
region of  Y  area  235  perimeter  118
slices  74
region of  W  area  1  perimeter  4
slices  4
region of  P  area  18  perimeter  28
slices  16
region of  E  area  34  perimeter  38
slices  22
region of  W  area  1  perimeter  4
slices  4
region of  C  area  2  perimeter  6
slices  4
region of  R  area  52  perimeter  54
slices  28
region of  O  area  1  perimeter  4
slices  4
region of  K  area  130  perimeter  78
slices  46
region of  M  area  330  perimeter  214
slices  134
region of  I  area  2  perimeter  6
slices  4
region of  I  area  55  perimeter  76
slices  32
region of  V  area  130  perimeter  80
slices  52
region of  E  area  48  perimeter  54
slices  36
region of  T  area  9  perimeter  18
slices  12
region of  W  area  84  perimeter  62
slices  38
region of  Q  area  66  perimeter  66
slices  44
region of  P  area  40  perimeter  38
slices  24
region of  B  area  1  perim

### --- Day 11: Plutonian Pebbles ---

The ancient civilization on Pluto was known for its ability to manipulate spacetime, and while The Historians explore their infinite corridors, you've noticed a strange set of physics-defying stones.

At first glance, they seem like normal stones: they're arranged in a perfectly straight line, and each stone has a number engraved on it.

The strange part is that every time you blink, the stones change.

Sometimes, the number engraved on a stone changes. Other times, a stone might split in two, causing all the other stones to shift over a bit to make room in their perfectly straight line.

As you observe them for a while, you find that the stones have a consistent behavior. Every time you blink, the stones each simultaneously change according to the first applicable rule in this list:

If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.
If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. (The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)
If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone.
No matter how the stones change, their order is preserved, and they stay on their perfectly straight line.

How will the stones evolve if you keep blinking at them? You take a note of the number engraved on each stone in the line (your puzzle input).

If you have an arrangement of five stones engraved with the numbers 0 1 10 99 999 and you blink once, the stones transform as follows:

The first stone, 0, becomes a stone marked 1.
The second stone, 1, is multiplied by 2024 to become 2024.
The third stone, 10, is split into a stone marked 1 followed by a stone marked 0.
The fourth stone, 99, is split into two stones marked 9.
The fifth stone, 999, is replaced by a stone marked 2021976.
So, after blinking once, your five stones would become an arrangement of seven stones engraved with the numbers 1 2024 1 0 9 9 2021976.

Here is a longer example:

```
Initial arrangement:
125 17

After 1 blink:
253000 1 7

After 2 blinks:
253 0 2024 14168

After 3 blinks:
512072 1 20 24 28676032

After 4 blinks:
512 72 2024 2 0 2 4 2867 6032

After 5 blinks:
1036288 7 2 20 24 4048 1 4048 8096 28 67 60 32

After 6 blinks:
2097446912 14168 4048 2 0 2 4 40 48 2024 40 48 80 96 2 8 6 7 6 0 3 2
```

In this example, after blinking six times, you would have 22 stones. After blinking 25 times, you would have 55312 stones!

Consider the arrangement of stones in front of you. How many stones will you have after blinking 25 times?

In [53]:
import copy

# Parse the input file
with open('input/day_11.txt') as f:
    lines = f.read().splitlines()

input = [int(x) for x in lines[0].split()]
output_blink = []

def split(element):
    return [int(str(element)[:len(str(element))//2]), int(str(element)[len(str(element))//2:])]


def blink(input_str):
    output = []
    for i in input_str:
        if i == 0:
            output.append(1)
        elif len(str(i)) % 2 == 0:
            s = str(i)
            output.append(int(s[:len(s)//2]))
            output.append(int(s[len(s)//2:]))
        else:
            output.append(i*2024)
    return output

iterations=25


for i in range(iterations):
    output_blink = blink(input)
    print('Iteration:', i+1,'Stones:', len(output_blink))
    input = output_blink

print('Number of stones after 25 blinks:', len(output_blink))




Iteration: 1 Stones: 11
Iteration: 2 Stones: 16
Iteration: 3 Stones: 24
Iteration: 4 Stones: 37
Iteration: 5 Stones: 54
Iteration: 6 Stones: 87
Iteration: 7 Stones: 139
Iteration: 8 Stones: 197
Iteration: 9 Stones: 288
Iteration: 10 Stones: 434
Iteration: 11 Stones: 676
Iteration: 12 Stones: 1059
Iteration: 13 Stones: 1524
Iteration: 14 Stones: 2348
Iteration: 15 Stones: 3608
Iteration: 16 Stones: 5422
Iteration: 17 Stones: 8422
Iteration: 18 Stones: 12504
Iteration: 19 Stones: 19017
Iteration: 20 Stones: 29143
Iteration: 21 Stones: 43763
Iteration: 22 Stones: 67531
Iteration: 23 Stones: 101506
Iteration: 24 Stones: 153592
Iteration: 25 Stones: 235850
Number of stones after 25 blinks: 235850


### --- Part Two ---

The Historians sure are taking a long time. To be fair, the infinite corridors are very large.

How many stones would you have after blinking a total of 75 times?

In [3]:
import math

# Parse the input file
with open('input/day_11.txt') as f:
    lines = f.read().splitlines()

l = [int(x) for x in lines[0].split()]
output_blink = []

def split(element):
    return [int(str(element)[:len(str(element))//2]), int(str(element)[len(str(element))//2:])]

def calc(mem, x, l):
    if l == 0:
        return 1
    if x in mem[l]:
        return mem[l][x]
    if x == 0:
        mem[l][x]=calc(mem,1,l-1)
    elif (int(math.log10(x))+1)%2==0:
        s = str(x)
        mem[l][x] = calc(mem, int(s[:len(s)//2]), l-1) + calc(mem, int(s[len(s)//2:]), l-1)
    else:
        mem[l][x]=calc(mem, x*2024, l-1)
    return mem[l][x]

limit = 76

mem = [{} for i in range(limit)]


for i in range(limit):
    print(sum([calc(mem,x,i) for x in l]))



8
11
16
24
37
54
87
139
197
288
434
676
1059
1524
2348
3608
5422
8422
12504
19017
29143
43763
67531
101506
153592
235850
354914
542670
822444
1243701
1903601
2873136
4376837
6658355
10063277
15362610
23272235
35350261
53815341
81447974
124080564
188308090
285687710
434938054
659141338
1002380413
1522945132
2310182312
3514780650
5332171354
8101306204
12313867401
18682359818
28405377321
43126623073
65491592662
99546014933
151085432732
229600870558
348736190360
529503860381
804721073582
1221772491781
1856059220475
2819665383645
4281367223324
6505390052367
9879254832498
15005447961611
22796809937586
34617680643180
52591631765528
79879301200500
121318828267874
184306261666737
279903140844645


### --- Day 10: Hoof It ---

You all arrive at a Lava Production Facility on a floating island in the sky. As the others begin to search the massive industrial complex, you feel a small nose boop your leg and look down to discover a reindeer wearing a hard hat.

The reindeer is holding a book titled "Lava Island Hiking Guide". However, when you open the book, you discover that most of it seems to have been scorched by lava! As you're about to ask how you can help, the reindeer brings you a blank topographic map of the surrounding area (your puzzle input) and looks up at you excitedly.

Perhaps you can help fill in the missing hiking trails?

The topographic map indicates the height at each position using a scale from 0 (lowest) to 9 (highest). For example:

```
0123
1234
8765
9876
```
Based on un-scorched scraps of the book, you determine that a good hiking trail is as long as possible and has an even, gradual, uphill slope. For all practical purposes, this means that a hiking trail is any path that starts at height 0, ends at height 9, and always increases by a height of exactly 1 at each step. Hiking trails never include diagonal steps - only up, down, left, or right (from the perspective of the map).

You look up from the map and notice that the reindeer has helpfully begun to construct a small pile of pencils, markers, rulers, compasses, stickers, and other equipment you might need to update the map with hiking trails.

A trailhead is any position that starts one or more hiking trails - here, these positions will always have height 0. Assembling more fragments of pages, you establish that a trailhead's score is the number of 9-height positions reachable from that trailhead via a hiking trail. In the above example, the single trailhead in the top left corner has a score of 1 because it can reach a single 9 (the one in the bottom left).

This trailhead has a score of 2:

```
...0...
...1...
...2...
6543456
7.....7
8.....8
9.....9
```

(The positions marked . are impassable tiles to simplify these examples; they do not appear on your actual topographic map.)

This trailhead has a score of 4 because every 9 is reachable via a hiking trail except the one immediately to the left of the trailhead:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

This topographic map contains two trailheads; the trailhead at the top has a score of 1, while the trailhead at the bottom has a score of 2:

```
10..9..
2...8..
3...7..
4567654
...8..3
...9..2
.....01
```

Here's a larger example:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

This larger example has 9 trailheads. Considering the trailheads in reading order, they have scores of 5, 6, 5, 3, 1, 3, 5, 3, and 5. Adding these scores together, the sum of the scores of all trailheads is 36.

The reindeer gleefully carries over a protractor and adds it to the pile. What is the sum of the scores of all trailheads on your topographic map?

In [6]:
# Parse the input file
with open('input/day_10.txt') as f:
    lines = f.read().splitlines()

alternatives={
    (-1,0),
    (0,1),
    (1,0),
    (0,-1)
}

mat = [[int(i) for i in x] for x in lines]

def dfs(mat, i,j,t, prev=-1):
    #print("dfs", i,j,prev)
    if i<0 or i>=len(mat) or j<0 or j>=len(mat[i]):
        return set()
    v = mat[i][j]
    if v!= prev+1:
        return set()
    if v == t:
        return {(i,j)}
    r = set()
    for a in alternatives:
       r=r.union(dfs(mat,i+a[0], j+a[1], t,v))
    return r

s = 0
for i in range(len(mat)):
    for j in range(len(mat[i])):
        if mat[i][j]==0:
            #print("call ",i,j)
            r=dfs(mat, i,j,9)
            #print("r",r)
            s+=len(r)
print(s)

652


### --- Part Two ---

The reindeer spends a few minutes reviewing your hiking trail map before realizing something, disappearing for a few minutes, and finally returning with yet another slightly-charred piece of paper.

The paper describes a second way to measure a trailhead called its rating. A trailhead's rating is the number of distinct hiking trails which begin at that trailhead. For example:

```
.....0.
..4321.
..5..2.
..6543.
..7..4.
..8765.
..9....
```

The above map has a single trailhead; its rating is 3 because there are exactly three distinct hiking trails which begin at that position:

```
.....0.   .....0.   .....0.
..4321.   .....1.   .....1.
..5....   .....2.   .....2.
..6....   ..6543.   .....3.
..7....   ..7....   .....4.
..8....   ..8....   ..8765.
..9....   ..9....   ..9....
```

Here is a map containing a single trailhead with rating 13:

```
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
```

This map contains a single trailhead with rating 227 (because there are 121 distinct hiking trails that lead to the 9 on the right edge and 106 that lead to the 9 on the bottom edge):

```
012345
123456
234567
345678
4.6789
56789.
```

Here's the larger example from before:

```
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
```

Considering its trailheads in reading order, they have ratings of 20, 24, 10, 4, 1, 4, 5, 8, and 5. The sum of all trailhead ratings in this larger example topographic map is 81.

You're not sure how, but the reindeer seems to have crafted some tiny flags out of toothpicks and bits of paper and is using them to mark trailheads on your topographic map. What is the sum of the ratings of all trailheads?

In [7]:
# Parse the input file
with open('input/day_10.txt') as f:
    lines = f.read().splitlines()


mat = [[int(i) for i in x] for x in lines]

alternatives={
    (-1,0),
    (0,1),
    (1,0),
    (0,-1)
}

def dfs(mat, i,j,t, prev=-1):
    #print("dfs", i,j,prev)
    if i<0 or i>=len(mat) or j<0 or j>=len(mat[i]):
        return 0
    v = mat[i][j]
    if v!= prev+1:
        return 0
    if v == t:
        return 1
    r = 0
    for a in alternatives:
       r+=dfs(mat,i+a[0], j+a[1], t,v)
    return r

#print(mat)
s = 0
for i in range(len(mat)):
    for j in range(len(mat[i])):
        if mat[i][j]==0:
            #print("call ",i,j)
            r=dfs(mat, i,j,9)
            #print("r",r)
            s+=r
print(s)

1432


### --- Day 9: Disk Fragmenter ---

Another push of the button leaves you in the familiar hallways of some friendly amphipods! Good thing you each somehow got your own personal mini submarine. The Historians jet away in search of the Chief, mostly by driving directly into walls.

While The Historians quickly figure out how to pilot these things, you notice an amphipod in the corner struggling with his computer. He's trying to make more contiguous free space by compacting all of the files, but his program isn't working; you offer to help.

He shows you the disk map (your puzzle input) he's already generated. For example:

2333133121414131402
The disk map uses a dense format to represent the layout of files and free space on the disk. The digits alternate between indicating the length of a file and the length of free space.

So, a disk map like 12345 would represent a one-block file, two blocks of free space, a three-block file, four blocks of free space, and then a five-block file. A disk map like 90909 would represent three nine-block files in a row (with no free space between them).

Each file on disk also has an ID number based on the order of the files as they appear before they are rearranged, starting with ID 0. So, the disk map 12345 has three files: a one-block file with ID 0, a three-block file with ID 1, and a five-block file with ID 2. Using one character for each block where digits are the file ID and . is free space, the disk map 12345 represents these individual blocks:

0..111....22222
The first example above, 2333133121414131402, represents these individual blocks:

00...111...2...333.44.5555.6666.777.888899
The amphipod would like to move file blocks one at a time from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks). For the disk map 12345, the process looks like this:

```
0..111....22222
02.111....2222.
022111....222..
0221112...22...
02211122..2....
022111222......
```

The first example requires a few more steps:

```
00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
```

The final step of this file-compacting process is to update the filesystem checksum. To calculate the checksum, add up the result of multiplying each of these blocks' position with the file ID number it contains. The leftmost block is in position 0. If a block contains free space, skip it instead.

Continuing the first example, the first few blocks' position multiplied by its file ID number are 0 * 0 = 0, 1 * 0 = 0, 2 * 9 = 18, 3 * 9 = 27, 4 * 8 = 32, and so on. In this example, the checksum is the sum of these, 1928.

Compact the amphipod's hard drive using the process he requested. What is the resulting filesystem checksum? (Be careful copy/pasting the input for this puzzle; it is a single, very long line.)

In [6]:
# Parse the input file
with open('input/day_9.txt') as f:
    lines = f.read().splitlines()

input = [int(x) for x in lines[0]]
# print(input)

sum = 0
block = 0
last = len(input)-1

# loop through the input
for i in range(len(input)):
    v=int(input[i])
    if i%2==0:
        for _ in (range(v)):
            sum+=((i//2)*block)
            block+=1
            #print((i//2), end='')
    elif last>i:
        for _ in range(v):
            while last>i and input[last]==0:
                last-=2
            if last<=i:
                break
            sum+=((last//2)*block)
            #print((last//2), end='')
            block+=1
            input[last]-=1
#print('')
print(sum)

6337367222422


### --- Part Two ---
Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?

The eager amphipod already has a new plan: rather than move individual blocks, he'd like to try compacting the files on his disk by moving whole files instead.

This time, attempt to move whole files to the leftmost span of free space blocks that could fit the file. Attempt to move each file exactly once in order of decreasing file ID number starting with the file with the highest file ID number. If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.

The first example from above now proceeds differently:

```
00...111...2...333.44.5555.6666.777.888899
0099.111...2...333.44.5555.6666.777.8888..
0099.1117772...333.44.5555.6666.....8888..
0099.111777244.333....5555.6666.....8888..
00992111777.44.333....5555.6666.....8888..
```

The process of updating the filesystem checksum is the same; now, this example's checksum would be 2858.

Start over, now compacting the amphipod's hard drive using this new method instead. What is the resulting filesystem checksum?

In [7]:
# Parse the input file
with open('input/day_9.txt') as f:
    lines = f.read().splitlines()

input = [int(x) for x in lines[0]]
# print(input)

disk = []

def find(v,i, a):
    print("find", v,i)
    for j in range(1,i,2):
        if a[j]>=v:
            print("return if", j, a[j])
            return j
    print("return f", i)
    return i

for i in range(len(input)):
    x = int(input[i])
    if i%2==0:
        disk.append((i//2, x))
    else:
        disk.append((-1, x))
#print(disk)
i = len(disk)-1
while i>0:
    v,s = disk[i]
    if v==-1:
        #empty space
        i-=1
        continue
    ni = -1
    for j in range(i):
        nv,ns = disk[j]
        if nv==-1 and ns>=s:
            ni = j
            break
    if ni==-1:
        i-=1
        continue
    _,ss = disk[ni]
    disk = disk[:ni]+[(v,s), (-1, ss-s)]+disk[ni+1:i]+[(-1,s)]+disk[i+1:]
#print(disk)
sum = 0
block = 0
for i in disk:
    v,s = i
    if v ==-1:
        block+=s
        continue
    for j in range(s):
        sum+= block*v
        block+=1
print(sum)

6361380647183


### --- Day 8: Resonant Collinearity ---

You find yourselves on the roof of a top-secret Easter Bunny installation.

While The Historians do their thing, you take a look at the familiar huge antenna. Much to your surprise, it seems to have been reconfigured to emit a signal that makes people 0.1% more likely to buy Easter Bunny brand Imitation Mediocre Chocolate as a Christmas gift! Unthinkable!

Scanning across the city, you find that there are actually many such antennas. Each antenna is tuned to a specific frequency indicated by a single lowercase letter, uppercase letter, or digit. You create a map (your puzzle input) of these antennas. For example:

```
............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
```

The signal only applies its nefarious effect at specific antinodes based on the resonant frequencies of the antennas. In particular, an antinode occurs at any point that is perfectly in line with two antennas of the same frequency - but only when one of the antennas is twice as far away as the other. This means that for any pair of antennas with the same frequency, there are two antinodes, one on either side of them.

So, for these two antennas with frequency a, they create the two antinodes marked with #:

```
..........
...#......
..........
....a.....
..........
.....a....
..........
......#...
..........
..........
```

Adding a third antenna with the same frequency creates several more antinodes. It would ideally add four antinodes, but two are off the right side of the map, so instead it adds only two:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......#...
..........
..........
```

Antennas with different frequencies don't create antinodes; A and a count as different frequencies. However, antinodes can occur at locations that contain antennas. In this diagram, the lone antenna with frequency capital A creates no antinodes but has a lowercase-a-frequency antinode at its location:

```
..........
...#......
#.........
....a.....
........a.
.....a....
..#.......
......A...
..........
..........
```

The first example has antennas with two different frequencies, so the antinodes they create look like this, plus an antinode overlapping the topmost A-frequency antenna:

```
......#....#
...#....0...
....#0....#.
..#....0....
....0....#..
.#....A.....
...#........
#......#....
........A...
.........A..
..........#.
..........#.
```

Because the topmost A-frequency antenna overlaps with a 0-frequency antinode, there are 14 total unique locations that contain an antinode within the bounds of the map.

Calculate the impact of the signal. How many unique locations within the bounds of the map contain an antinode?

In [4]:

# Parse the input file
with open('input/day_8.txt') as f:
    lines = f.read().splitlines()


def valid(x, mat):
    return 0 <= x[0] < len(mat) and 0 <= x[1] < len(mat[x[0]])

antennas={}
for i in range(len(lines)):
    for j in range(len(lines[i])):
        v = lines[i][j]
        if v!='.':
            if v not in antennas:
                antennas[v]=[]
            antennas[v].append((i,j))

antinodes=set()

for k,v in antennas.items():
    for a in v:
        for b in v:
            if a==b:
                continue
            dx=a[0]-b[0]
            dy=a[1]-b[1]

            nx=b[0]-dx
            ny=b[1]-dy

            antinodes.add((nx,ny))
print(sum([1 for x in antinodes if valid(x, lines)]))

247


### --- Part Two ---

Watching over your shoulder as you work, one of The Historians asks if you took the effects of resonant harmonics into your calculations.

Whoops!

After updating your model, it turns out that an antinode occurs at any grid position exactly in line with at least two antennas of the same frequency, regardless of distance. This means that some of the new antinodes will occur at the position of each antenna (unless that antenna is the only one of its frequency).

So, these three T-frequency antennas now create many antinodes:

```
T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........
```

In fact, the three T-frequency antennas are all exactly in line with two antennas, so they are all also antinodes! This brings the total number of antinodes in the above example to 9.

The original example now has 34 antinodes, including the antinodes that appear on every antenna:

```
##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##
```

Calculate the impact of the signal using this updated model. How many unique locations within the bounds of the map contain an antinode?

In [5]:

# Parse the input file
with open('input/day_8.txt') as f:
    lines = f.read().splitlines()

def valid(x, mat):
    return 0 <= x[0] < len(mat) and 0 <= x[1] < len(mat[x[0]])

antennas={}
for i in range(len(lines)):
    for j in range(len(lines[i])):
        v = lines[i][j]
        if v!='.':
            if v not in antennas:
                antennas[v]=[]
            antennas[v].append((i,j))
antinodes=set()
for k,v in antennas.items():
    for a in v:
        for b in v:
            if a==b:
                continue
            dx=a[0]-b[0]
            dy=a[1]-b[1]

            nx=b[0]
            ny=b[1]
            while valid((nx,ny),lines):
                antinodes.add((nx,ny))
                nx=nx-dx
                ny=ny-dy

print(len(antinodes))

861


### --- Day 7: Bridge Repair ---

The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```

Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (+) and multiply (*).

Only three of the above equations can be made true by inserting operators:

190: 10 19 has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
3267: 81 40 27 has two positions for operators. Of the four possible configurations of the operators, two cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
292: 11 6 16 20 can be solved in exactly one way: 11 + 6 * 16 + 20.
The engineers just need the total calibration result, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is 3749.

Determine which equations could possibly be true. What is their total calibration result?

In [41]:
# Parse the input file
with open('input/day_7.txt') as f:
    lines = f.read().splitlines()

formula_dict = {}

def isvalid(test_value, numbers):
    if numbers[0] > test_value:
        return False
    if len(numbers) == 1:
        return numbers[0] == test_value
    if isvalid(test_value, [numbers[0] + numbers[1]] + numbers[2:]):
        return True
    elif isvalid(test_value, [numbers[0] * numbers[1]] + numbers[2:]):
        return True
    return False

def solve_equation(lines) -> dict:
    output = 0
    for line in lines:
        test_value, numbers = line.split(":")
        test_value = int(test_value)
        numbers = list(map(int, numbers.strip().split()))
        # print(test_value, numbers)
        if isvalid(test_value, numbers):
            output += int(test_value)
    return  output

sum_of_solvable = solve_equation(lines)

print('Sum of solvable equations:', sum_of_solvable)

Sum of solvable equations: 8401132154762


### --- Part Two ---

The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The concatenation operator (||) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

156: 15 6 can be made true through a single concatenation: 15 || 6 = 156.
7290: 6 8 6 15 can be made true using 6 * 8 || 6 * 15.
192: 17 8 14 can be made true using 17 || 8 + 14.
Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new total calibration result of 11387.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. What is their total calibration result?

In [43]:
# Parse the input file
with open('input/day_7.txt') as f:
    lines = f.read().splitlines()

formula_dict = {}

def isvalid(test_value, numbers):
    if numbers[0] > test_value:
        return False
    if len(numbers) == 1:
        return numbers[0] == test_value
    if isvalid(test_value, [numbers[0] + numbers[1]] + numbers[2:]):
        return True
    elif isvalid(test_value, [numbers[0] * numbers[1]] + numbers[2:]):
        return True
    elif isvalid(test_value, [int(str(numbers[0]) + str(numbers[1]))] + numbers[2:]):
        return True
    return False


def solve_equation(lines) -> dict:
    output = 0
    for line in lines:
        test_value, numbers = line.split(":")
        test_value = int(test_value)
        numbers = list(map(int, numbers.strip().split()))
        # print(test_value, numbers)
        if isvalid(test_value, numbers):
            output += int(test_value)
    return  output

sum_of_solvable = solve_equation(lines)

print('Sum of solvable equations:', sum_of_solvable)

Sum of solvable equations: 95297119227552


### --- Day 6: Guard Gallivant ---

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
```

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.
Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```

Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```

This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```

By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:

```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```

In this example, the guard will visit 41 distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

In [10]:
DIRECTIONS = ("^", "v", "<", ">")
TURN_90 = {"^": ">", "v": "<", "<": "^", ">": "v"}
TO_ADD = {
    "^": (-1, 0),
    "v": (1, 0),
    ">": (0, 1),
    "<": (0, -1)
}

def get_guard_position(guard_map):
    for i in range(len(guard_map)):
        for j in range(len(guard_map[i])):
            if guard_map[i][j] in DIRECTIONS:
                return (i, j), guard_map[i][j]
    return -1, -1

def get_obstacles(guard_map):
    obstacles = []
    for i in range(len(guard_map)):
        for j in range(len(guard_map[i])):
            if guard_map[i][j] == "#":
                obstacles.append((i, j))
    return obstacles

def is_outside(pos, dir, rows, cols):
    x, y = pos
    return any([
        (dir == "^" and x == 0),
        (dir == "v" and x == rows - 1),
        (dir == "<" and y == 0),
        (dir == ">" and y == cols - 1)
    ])

def add_tuple(A, B):
    return tuple([A[i] + B[i] for i in range(len(A))])

def add_pos(pos, dir):
    return add_tuple(pos, TO_ADD[dir])

def solve(guard_map, animate=False):

    # get the grid size
    rows, cols = len(guard_map), len(guard_map[0])

    # get the obstacles positions
    obstacles = get_obstacles(guard_map)
    visited_obstacles = []

    # get the guard position and direction
    pos, dir = get_guard_position(guard_map)

    positions = [pos]
    is_loop = False
    frame = 0

    while not is_outside(pos, dir, rows, cols):
        next_pos = add_pos(pos, dir)

        x, y = pos
        guard_map[x][y] = " "

        if next_pos in obstacles:

            if (next_pos, dir) in visited_obstacles:
                is_loop = True
                break

            visited_obstacles.append((next_pos, dir))

            dir = TURN_90[dir]
            next_pos = pos
            continue

        pos = next_pos

        if pos not in positions:
            positions.append(pos)

        # Animation
        if animate:
            x, y = pos
            guard_map[x][y] = dir
            M = ""
            for r in guard_map:
                M = M + "".join(r) + "\n"

            with open(f"output/day_6/{frame}.txt", "w") as f:
                f.write(M)

            frame += 1
        # End Animation

    return len(positions), positions, is_loop


# Parse the input file
with open('input/day_6.txt') as f:
    lines = f.readlines()
    clean_lines = [list(x.replace("\n", "")) for x in lines]

ans1, positions, _ = solve(clean_lines, True)

print(ans1)

5269


--- Part Two ---


While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

```
....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
```

Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
```

Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
```

Option four, put an alchemical retroencabulator near the bottom left corner:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
```

Option five, put the alchemical retroencabulator a bit to the right instead:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
```

Option six, put a tank of sovereign glue right next to the tank of universal solvent:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
```

It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?

In [35]:
from sys import argv
import time

t_start = time.process_time()

directions = [(0,-1), (1,0), (0,1), (-1,0)]

with open('input/day_6.txt', "r") as f:
    grid = [[c for c in l.strip()] for l in f]

print(f"reading data: {(time.process_time()-t_start)*1000:.3F} ms")

t_lap = time.process_time()

width, height = (len(grid[0]), len(grid))

def outside(x,y):
    return not (0<=x<width and 0<=y<height)

neighbor = dict()
blocks = set()
nodes = [(x, y, d) for x in range(width) for y in range(height) for d in range(4)]

for x,y in [(x,y) for x in range(width) for y in range(height)]:
    if grid[y][x] == "#":
        blocks.add((x,y))
    elif grid[y][x] == "^":
        startpos = (x,y,0)

for x,y,d in nodes:
    if outside(x+directions[d][0], y+directions[d][1]):
        neighbor[(x,y,d)] = (-1,-1,-1)
    elif (x+directions[d][0], y+directions[d][1]) in blocks:
        neighbor[(x,y,d)] = (x,y,(d+1)%4)
    else:
        neighbor[(x,y,d)] = (x+directions[d][0], y+directions[d][1], d)

print(f"processing nodes: {(time.process_time()-t_lap)*1000:.3F} ms")
t_lap = time.process_time()

orig_path = []
orig_visited = set()

pos = startpos

while not pos==(-1,-1,-1):
    orig_path.append(pos)
    orig_visited.add((pos[0],pos[1]))
    pos = neighbor[pos]

print(f"walking path: {(time.process_time()-t_lap)*1000:.3F} ms")
print(f"path has {len(orig_path)} steps on {len(orig_visited)} unique nodes")

loop_obstacles = set()
non_loop_obstacles = set()
for index, pos in enumerate(orig_path[1:],1):
    x,y = (pos[0],pos[1])
    if (x,y) in loop_obstacles or (x,y) in non_loop_obstacles:
        continue
    for d in range(4):
        prev = (x-directions[d][0], y-directions[d][1], d)
        neighbor[prev] = (prev[0], prev[1], (d+1)%4)
    visited = set()
    cur = orig_path[index-1]
    while not cur==(-1,-1,-1) and cur not in visited:
        visited.add(cur)
        cur = neighbor[cur]
    if cur==(-1,-1,-1):
        non_loop_obstacles.add((pos[0],pos[1]))
    else:
        loop_obstacles.add((pos[0],pos[1]))
    for d in range(4):
        prev = (x-directions[d][0], y-directions[d][1], d)
        neighbor[prev] = (pos[0],pos[1],d)

t_round1 = time.process_time()
print(f"checking for obstacles: {(t_round1-t_lap)*1000:.3F} ms")
print(f"total time elapsed: {(t_round1-t_start)*1000:.3F} ms")
print(f"{len(loop_obstacles)} obstacles lead to a loop. {len(non_loop_obstacles)} don't. {len(loop_obstacles)+len(non_loop_obstacles)} positions investigated")

print(f"start list-based approach")
t_lap = time.process_time()

neighbor = [[[(x,y,d) for d in range(4)] for y in range(height)] for x in range(width)]
print(f"neighbor array: {len(neighbor)} x {len(neighbor[0])} x {len(neighbor[0][0])}")
blocks = [[False for y in range(height)] for x in range(width)]
nodes = [(x, y, d) for x in range(width) for y in range(height) for d in range(4)]

for x,y in [(x,y) for x in range(width) for y in range(height)]:
    if grid[y][x] == "#":
        blocks[x][y] = True
    elif grid[y][x] == "^":
        startpos = (x,y,0)

for x,y,d in nodes:
    if outside(x+directions[d][0], y+directions[d][1]):
        neighbor[x][y][d] = (-1,-1,-1)
    elif blocks[x+directions[d][0]][y+directions[d][1]]:
        neighbor[x][y][d] = (x,y,(d+1)%4)
    else:
        neighbor[x][y][d] = (x+directions[d][0], y+directions[d][1], d)

print(f"processing nodes: {(time.process_time()-t_lap)*1000:.3F} ms")
t_lap = time.process_time()

orig_path = []
orig_visited = [[False for y in range(height)] for x in range(width)]

pos = startpos

while not pos==(-1,-1,-1):
    x,y,d = pos
    orig_path.append(pos)
    orig_visited[x][y] = True
    pos = neighbor[x][y][d]

print(f"walking path: {(time.process_time()-t_lap)*1000:.3F} ms")
print(f"path has {len(orig_path)} steps on {sum(sum(row) for row in orig_visited)} unique nodes")

loop_obstacles = [[False for y in range(height)] for x in range(width)]
non_loop_obstacles = [[False for y in range(height)] for x in range(width)]
visited = [[[0 for d in range(4)] for y in range(height)] for x in range(width)]
for index, pos in enumerate(orig_path[1:],1):
    x,y = (pos[0],pos[1])
    if loop_obstacles[x][y] or non_loop_obstacles[x][y]:
        continue
    for d in range(4):
        prev = (x-directions[d][0], y-directions[d][1], d)
        if outside(prev[0], prev[1]): continue
        neighbor[prev[0]][prev[1]][d] = (prev[0], prev[1], (d+1)%4)

    cur = orig_path[index-1]
    while not cur==(-1,-1,-1) and not visited[cur[0]][cur[1]][cur[2]] == index:
        visited[cur[0]][cur[1]][cur[2]] = index
        cur = neighbor[cur[0]][cur[1]][cur[2]]
    if cur==(-1,-1,-1):
        non_loop_obstacles[x][y] = True
    else:
        loop_obstacles[x][y] = True
    for d in range(4):
        prev = (x-directions[d][0], y-directions[d][1], d)
        if outside(prev[0], prev[1]): continue
        neighbor[prev[0]][prev[1]][d] = (pos[0],pos[1],d)


t_round2 = time.process_time()
print(f"checking for obstacles: {(t_round2-t_lap)*1000:.3F} ms")
print(f"total time elapsed: {(t_round2-t_round1)*1000:.3F} ms")
loop, non_loop = map (lambda l: sum(sum(row) for row in l), (loop_obstacles, non_loop_obstacles))
print(f"{loop} obstacles lead to a loop. {non_loop} don't. {loop+non_loop} positions investigated")

reading data: 0.000 ms
processing nodes: 78.125 ms
walking path: 0.000 ms
path has 6026 steps on 5269 unique nodes
checking for obstacles: 2421.875 ms
total time elapsed: 2500.000 ms
1957 obstacles lead to a loop. 3312 don't. 5269 positions investigated
start list-based approach
neighbor array: 130 x 130 x 4
processing nodes: 156.250 ms
walking path: 0.000 ms
path has 6026 steps on 5269 unique nodes
checking for obstacles: 2984.375 ms
total time elapsed: 3140.625 ms
1957 obstacles lead to a loop. 3312 don't. 5269 positions investigated


### --- Day 5: Print Queue ---

Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

```
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13
```
```
75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
```
The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
53 is correctly fourth because it is before page number 29 (53|29).
29 is the only page left and so is correctly last.
Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

```
75,47,61,53,29
97,61,53,29,13
75,29,13
```

These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?

In [36]:
# Parse the input file
with open('input/day_5.txt') as f:
    lines = f.read().splitlines()

def parse_rules(rules):
    rules_combined = {}
    # flatten multiple rules into a single list while preserving the order
    for rule in rules:
        left, right = [int(x) for x in rule.split('|')]
        if left not in rules_combined:
            rules_combined[left] = set()
        rules_combined[left].add(right)
    return rules_combined

def parse_updates(updates, parsed_rules):
    sum = 0
    for update in updates:
        update = [int(x) for x in update.split(',')]
        sum += process_print(parsed_rules, update)
    print(sum)

def process_print(parsed_rules, record):
    read = set()
    # check if the rules are violated
    for n in record:
        if n in parsed_rules and len(parsed_rules[n].intersection(read)) > 0:
            return 0
        read.add(n)
    return record[len(record) // 2]


# code to split the input into rules and updates
rules = []
updates = []
for line in lines:
    if '|' in line:
        rules.append(line)
    elif '' != line:
        updates.append(line)

# print(rules)
# print(updates)

parsed_rule = parse_rules(rules)
parsed_updates = parse_updates(updates,parsed_rule)

4569


In [9]:
# Parse the input file
with open('input/day_5.txt') as f:
    lines = f.read().splitlines()

def parse_rules(rules):
    rules_combined = {}
    # flatten multiple rules into a single list while preserving the order
    for rule in rules:
        left, right = [int(x) for x in rule.split('|')]
        if left not in rules_combined:
            rules_combined[left] = set()
        rules_combined[left].add(right)
    return rules_combined

def parse_updates(updates, parsed_rules):
    sum = 0
    for update in updates:
        update = [int(x) for x in update.split(',')]
        sum += process_print(parsed_rules, update)
    print(sum)

def process_print(parsed_rules, record):
    read = set()
    # check if the rules are violated
    for n in record:
        if n in parsed_rules and len(parsed_rules[n].intersection(read)) > 0:
            return 0
        read.add(n)
    return record[len(record) // 2]


# code to split the input into rules and updates
rules = []
updates = []
for line in lines:
    if '|' in line:
        rules.append(line)
    elif '' != line:
        updates.append(line)

# print(rules)
# print(updates)

parsed_rule = parse_rules(rules)
parsed_updates = parse_updates(updates,parsed_rule)

4569


--- Part Two ---

While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

75,97,47,61,53 becomes 97,75,47,61,53.
61,13,29 becomes 61,29,13.
97,13,75,29,47 becomes 97,75,47,29,13.
After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?

In [16]:
# Parse the input file
with open('input/day_5.txt') as f:
    lines = f.read().splitlines()


def findInvalid(record,i, rules):
    # Find the invalid record
    v = record[i]
    if v not in rules:
        return -1
    for j in range(i):
        if record[j] in rules[v]:
            return j
    return -1


def sort(record, rules, step=0):
    # Sort the invalid record as per the rules
    p=-1
    for i in range(len(record)):
        p = findInvalid(record, i, rules)
        if p!=-1:
            record = record[:p]+[record[i]]+record[p:i]+record[i+1:]
            break
    if p==-1:
        return record
    return sort(record, rules, step+1)

def process_print(parsed_rules, record):
    read = set()
    # check if the rules are violated
    for n in record:
        if n in parsed_rules and len(parsed_rules[n].intersection(read)) > 0:
            corrected_record = sort(record, parsed_rules)
            return corrected_record[len(corrected_record) // 2]
        read.add(n)
    return 0


def parse_rules(rules):
    # flatten multiple rules into a single list while preserving the order
    rules_combined = {}
    for rule in rules:
        left, right = [int(x) for x in rule.split('|')]
        if left not in rules_combined:
            rules_combined[left] = set()
        rules_combined[left].add(right)
    return rules_combined

def parse_updates(updates, parsed_rules):
    # Function to parse the updates
    sum = 0
    for update in updates:
        update = [int(x) for x in update.split(',')]
        sum += process_print(parsed_rules, update)
    print(sum)




# code to split the input into rules and updates
rules = []
updates = []
for line in lines:
    if '|' in line:
        rules.append(line)
    elif '' != line:
        updates.append(line)

# print(rules)
# print(updates)

parsed_rule = parse_rules(rules)
parsed_updates = parse_updates(updates,parsed_rule)

6456


### --- Day 4: Ceres Search ---
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

The actual word search will be full of letters instead. For example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```
Take a look at the little Elf's word search. How many times does XMAS appear?

In [27]:
def find_word(grid, word):
    def search(x, y, dx, dy):
        for i in range(len(word)):
            if not (0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == word[i]):
                return False
            x += dx
            y += dy
        return True

    directions = [(0, 1), (1, 0), (1, 1), (1, -1), (0, -1), (-1, 0), (-1, -1), (-1, 1)]
    count = 0

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            for dx, dy in directions:
                if search(i, j, dx, dy):
                    count += 1

    return count

# Parse the input file
with open('input/day_4.txt') as f:
    lines = f.read().splitlines()

# Example usage
word = "XMAS"
print(find_word(lines, word))  # Output: 18

18


--- Part Two ---
The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an XMAS puzzle; it's an X-MAS puzzle in which you're supposed to find two MAS in the shape of an X. One way to achieve that is like this:

```
M.S
.A.
M.S
```
Irrelevant characters have again been replaced with . in the above diagram. Within the X, each MAS can be written forwards or backwards.

Here's the same example from before, but this time all of the X-MASes have been kept instead:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```

In this example, an X-MAS appears 9 times.

Flip the word search from the instructions back over to the word search side and try again. How many times does an X-MAS appear?

In [29]:
def find_x_mas_pattern(lines):

    # Check if the cell is part of an X-MAS pattern
    m = len(lines)
    n = len(lines[0])

    def check(r, c):
        if lines[r][c] != 'A':
            return False
        ul = lines[r-1][c-1]
        ur = lines[r-1][c+1]
        dl = lines[r+1][c-1]
        dr = lines[r+1][c+1]
        return sorted([ul, ur, dl, dr]) == ['M', 'M', 'S', 'S'] and ul != dr

    return sum(check(r, c) for r in range(1, m-1) for c in range(1, n-1))

# Parse the input file
with open('input/day_4.txt') as f:
    lines = f.read().splitlines()

print(find_x_mas_pattern(lines))

1854


### --- Day 3: Mull It Over ---

"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like `mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 )` do nothing.

For example, consider the following section of corrupted memory:

`xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))`
Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (2*4 + 5*5 + 11*8 + 8*5).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?

In [12]:
import re

sum = 0

# Define the pattern to match the mul instructions
pattern = re.compile(r'mul\((\d+),(\d+)\)')

# Parse the input file
with open('input/day_3.txt') as f:
    lines = f.readlines()

for line in lines:

    matches = pattern.findall(line)

    extracted_parts = [(int(x), int(y)) for x, y in matches]

    # print(extracted_parts)

    for part in extracted_parts:
        sum += part[0] * part[1]

print('Sum:', sum)


Sum: 170778545


### --- Part Two ---

As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

- The do() instruction enables future mul instructions.
- The don't() instruction disables future mul instructions.
Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

`xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))`

This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (2*4 + 8*5).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?

In [21]:
import re

sum = 0
enabled = True


# Define the pattern to match the mul instructions
pattern = re.compile(r'(do\(\)|don\'t\(\)|mul\((\d+),(\d+)\))')

# Parse the input file
with open('input/day_3.txt') as f:
    lines = f.readlines()

for line in lines:
    # print(line)

    # Find all matches in the input string
    matches = pattern.findall(line)
    # print(matches)

    for match in matches:
        if match[0] == "do()":
            enabled = True
        elif match[0] == "don't()":
            enabled = False
        elif "mul" in match[0] and enabled:
            sum += int(match[1]) * int(match[2])

print('Sum:', sum)

Sum: 82868252


### --- Day 2: Red-Nosed Reports ---
Fortunately, the first location The Historians want to search isn't a long walk from the Chief Historian's office.

While the Red-Nosed Reindeer nuclear fusion/fission plant appears to contain no sign of the Chief Historian, the engineers there run up to you as soon as they see you. Apparently, they still talk about the time Rudolph was saved through molecular synthesis from a single electron.

They're quick to add that - since you're already here - they'd really appreciate your help analyzing some unusual data from the Red-Nosed reactor. You turn to check if The Historians are waiting for you, but they seem to have already divided into groups that are currently searching every corner of the facility. You offer to help with the unusual data.

The unusual data (your puzzle input) consists of many reports, one report per line. Each report is a list of numbers called levels that are separated by spaces. For example:

```python
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```


This example data contains six reports each containing five levels.

The engineers are trying to figure out which reports are safe. The Red-Nosed reactor safety systems can only tolerate levels that are either gradually increasing or gradually decreasing. So, a report only counts as safe if both of the following are true:

- The levels are either all increasing or all decreasing.
- Any two adjacent levels differ by at least one and at most three.

In the example above, the reports can be found safe or unsafe by checking those rules:

- 7 6 4 2 1: Safe because the levels are all decreasing by 1 or 2.
- 1 2 7 8 9: Unsafe because 2 7 is an increase of 5.
- 9 7 6 2 1: Unsafe because 6 2 is a decrease of 4.
- 1 3 2 4 5: Unsafe because 1 3 is increasing but 3 2 is decreasing.
- 8 6 4 4 1: Unsafe because 4 4 is neither an increase or a decrease.
- 1 3 6 7 9: Safe because the levels are all increasing by 1, 2, or 3.

So, in this example, 2 reports are safe.

Analyze the unusual data from the engineers. How many reports are safe?

In [7]:
safe_count=0
unsafe_count=0

# Parse the input file
with open('input/day_2.txt') as f:
    lines = f.readlines()
levels = [list(map(int, line.split())) for line in lines]
#print(levels)

# Check each report
for report in levels:
    #print('Report:', report)
    safe_flag = True
    score = int()
    increasing = None

    # Check each level in the report with the previous level
    for previousLevel, currentLevel in zip(report, report[1:]):
        # print('Previous:', previousLevel, 'Current:', currentLevel)

        score = currentLevel - previousLevel

        if score < -3 or score > 3 or score == 0:
            safe_flag=False
        if increasing and score < 0:
            safe_flag=False
        if increasing == False and score > 0:
            safe_flag=False

        if score < 0:
            increasing = False
        else:
            increasing = True

    if safe_flag:
        safe_count += 1
    else:
        unsafe_count += 1

print('Safe:', safe_count, 'Unsafe:', unsafe_count)

Safe: 479 Unsafe: 521


### --- Part Two ---

The engineers are surprised by the low number of safe reports until they realize they forgot to tell you about the Problem Dampener.

The Problem Dampener is a reactor-mounted module that lets the reactor safety systems tolerate a single bad level in what would otherwise be a safe report. It's like the bad level never happened!

Now, the same rules apply as before, except if removing a single level from an unsafe report would make it safe, the report instead counts as safe.

More of the above example's reports are now safe:

- 7 6 4 2 1: Safe without removing any level.
- 1 2 7 8 9: Unsafe regardless of which level is removed.
- 9 7 6 2 1: Unsafe regardless of which level is removed.
- 1 3 2 4 5: Safe by removing the second level, 3.
- 8 6 4 4 1: Safe by removing the third level, 4.
- 1 3 6 7 9: Safe without removing any level.

Thanks to the Problem Dampener, 4 reports are actually safe!

Update your analysis by handling situations where the Problem Dampener can remove a single level from unsafe reports. How many reports are now safe?

In [11]:
safe_count=0
unsafe_count=0

def checkSafe(report):

    increasing = None

    # Check each level in the report with the previous level
    for previousLevel, currentLevel in zip(report, report[1:]):
        # print('Previous:', previousLevel, 'Current:', currentLevel)

        score = currentLevel - previousLevel

        if score < -3 or score > 3 or score == 0:
            return False
        if increasing and score < 0:
             return False
        if increasing == False and score > 0:
             return False

        if score < 0:
            increasing = False
        else:
            increasing = True

    return True

# Parse the input file
with open('input/day_2.txt') as f:
    lines = f.readlines()
levels = [list(map(int, line.split())) for line in lines]
#print(levels)

for report in levels:
    if checkSafe(report):
        safe_count += 1
    else: # Part 2
        for i, level in enumerate(report):
            reportCopy = report.copy()
            reportCopy.pop(i)
            if checkSafe(reportCopy):
                safe_count += 1
                break

print(safe_count)

531


### --- Day 1: Historian Hysteria ---

Upon pouring into the office, everyone confirms that the Chief Historian is indeed nowhere to be found. Instead, the Elves discover an assortment of notes and lists of historically significant locations! This seems to be the planning the Chief Historian was doing before he left. Perhaps these notes can be used to determine which locations to search?

Throughout the Chief's office, the historically significant locations are listed not by name but by a unique number called the location ID. To make sure they don't miss anything, The Historians split into two groups, each searching the office and trying to create their own complete list of location IDs.

There's just one problem: by holding the two lists up side by side (your puzzle input), it quickly becomes clear that the lists aren't very similar. Maybe you can help The Historians reconcile their lists?

For example:

```python
3   4
4   3
2   5
1   3
3   9
3   3
```
Maybe the lists are only off by a small amount! To find out, pair up the numbers and measure how far apart they are. Pair up the smallest number in the left list with the smallest number in the right list, then the second-smallest left number with the second-smallest right number, and so on.

Within each pair, figure out how far apart the two numbers are; you'll need to add up all of those distances. For example, if you pair up a 3 from the left list with a 7 from the right list, the distance apart is 4; if you pair up a 9 with a 3, the distance apart is 6.

In the example list above, the pairs and distances would be as follows:

- The smallest number in the left list is 1, and the smallest number in the right list is 3. The distance between them is 2.
- The second-smallest number in the left list is 2, and the second-smallest number in the right list is another 3. The distance between them is 1.
- The third-smallest number in both lists is 3, so the distance between them is 0.
- The next numbers to pair up are 3 and 4, a distance of 1.
- The fifth-smallest numbers in each list are 3 and 5, a distance of 2.
- Finally, the largest number in the left list is 4, while the largest number in the right list is 9; these are a distance 5 apart.

To find the total distance between the left list and the right list, add up the distances between all of the pairs you found. In the example above, this is 2 + 1 + 0 + 1 + 2 + 5, a total distance of 11!

Your actual left and right lists contain many location IDs. What is the total distance between your lists?

In [1]:
list1 = []
list2 = []
total_sum = 0
with open('input/day_1.txt') as f:
    lines = f.readlines()
    for line in lines:
        values = line.split()
        list1.append(int(values[0]))
        list2.append(int(values[1]))
    list1.sort()
    list2.sort()
    total_sum = sum(abs(a - b) for a, b in zip(list1, list2))
print(total_sum)

2176849


### --- Part Two ---


Your analysis only confirmed what everyone feared: the two lists of location IDs are indeed very different.

Or are they?

The Historians can't agree on which group made the mistakes or how to read most of the Chief's handwriting, but in the commotion you notice an interesting detail: a lot of location IDs appear in both lists! Maybe the other numbers aren't location IDs at all but rather misinterpreted handwriting.

This time, you'll need to figure out exactly how often each number from the left list appears in the right list. Calculate a total similarity score by adding up each number in the left list after multiplying it by the number of times that number appears in the right list.

Here are the same example lists again:

```python
3   4
4   3
2   5
1   3
3   9
3   3
```
For these example lists, here is the process of finding the similarity score:

- The first number in the left list is 3. It appears in the right list three times, so the similarity score increases by 3 * 3 = 9.
- The second number in the left list is 4. It appears in the right list once, so the similarity score increases by 4 * 1 = 4.
- The third number in the left list is 2. It does not appear in the right list, so the similarity score does not increase (2 * 0 = 0).
- The fourth number, 1, also does not appear in the right list.
- The fifth number, 3, appears in the right list three times; the similarity score increases by 9.
- The last number, 3, appears in the right list three times; the similarity score again increases by 9.

So, for these example lists, the similarity score at the end of this process is 31 (9 + 4 + 0 + 0 + 9 + 9).

Once again consider your left and right lists. What is their similarity score?

In [3]:
from collections import Counter

list1 = []
list2 = []
total_sum = 0
with open('input/day_1.txt') as f:
    lines = f.readlines()
    for line in lines:
        values = line.split()
        list1.append(int(values[0]))
        list2.append(int(values[1]))
    lc  = Counter(list2)
    for l in list1:
        total_sum += l * lc[l]
print(total_sum)

23384288
